# String Processing with aq_pp

In this notebook, we'll go over string processing techniques using `aq_pp` command's option and builtin function. We'll start with basic usage of each option and function, then move towards to the applications.


    
## Objective
By the end of this notebook, reader should be able to

1. understand purpose and correct usage of `-map(c/f)` option and some of the `aq-emod`'s functions for string manipulate.
2. understand and use MapFrom/MapTo syntax
3. perform basic level of string manipulation using the tools introduced

## Pre-requsites
Readers are assumed to be equipped with decent amount of knowledge of the followings.
- `aq_pp`
- `aq_input`
- `aq_output`
- Regex

## Overview

There are 2 major ways of manipulating and processing strings, using
1. `aq_pp`'s option, `-map`, `-mapf`, `-mapc`
2. [Data extraction function](http://auriq.com/documentation/source/reference/manpages/aq-emod.html?highlight=emod#data-extraction-and-encode-decode-functions) from aq-emod.

### `-Map(c/f)`

There are 3 types of map options, namely
- `-map`: extract desired pattern(s) from given column, and map it back to the same column based on MapTo string.
- `-mapf` & `-mapc`: these are used in pairs. `-mapf` specifies the source column to extract patterns from, and `-mapc` is used to specify destination column and mapping pattern.

**Syntax**<br>

**Map**<br>
```aq_pp ... -map[,AtrLst] colName "MapFrom" "MapTo"```

**Mapf/Mapc**<br>
```aq_pp ... -mapf,[AtrLst] ColName MapFrom -mapc ColSpec|ColName MapTo```

Where
- `[,AtrLst]`: list of attributes to provide. 
    * `ncas` for case insensitive match or you can provide 
    * [regular expression attributes](http://auriq.com/documentation/source/reference/manpages/aq-emod.html#regex-attributes) to specify the type of regex being used.
- `colName`: column name
- `ColSpec`: column spec, in case of storing the result in a new column.





## Data

We'll be using airline flight search history data (modified from the original for privacy purpose.) This dataset contains long string columns that are perfect for this tutorial.
Below is what the data looks like.

search|
-----|
From=HNDTo=NGODate=20150506Class=Y|
From=NGik0To=OKADate=20150425Class=1|
From=OKATo=NGODate=20l5o4A5Class=Y|
From=OKATo=NGODate=20150425Class=S|
From=OKATo=nG0ODate=20150425Class=3
From=3GOTo=OKADate=S0150F25Class=Y|
From=NGOTo=OKADate=20150419Class=Y|
From=OKaTo=NGODate=20150517Class=Y|


Each rows are record for a one search query, and from left to right, composted of
- `From=*`: 3 capital letter alphabetical airport code
- `To=*`: 3 capital letter alphabetical airport code
- `Date=*`: Date of the flight
- `Class=*`: class of flight, categorical value

Note that some row has invalid entries. 


### Common Rules / Concepts
There are several concepts to be covered that applies to both `-map(c/f)` options as well as `aq-emod` functions.

**1. Extraction / Mapping Pattern Rules**
There are 3 types of pattern matching string rule used in `aq_pp`, 2 for extraction and 1 for mapping.
1. Extraction: [Regex](https://www.regular-expressions.info/) and [MapFrom](http://auriq.com/documentation/source/reference/manpages/aq_pp.html#rt-mapfrom-syntax) - these are used to specify patterns you'd like to extract from a given string.
2. Mapping: [MapTo](http://auriq.com/documentation/source/reference/manpages/aq_pp.html#mapto-syntax) - specify pattern to map the extracted string to destination

Both of MapFrom & MapTo will be covered with concrete examples.

**2. Common Syntactic Rules**
1. source and destination columns for arguments needs to be STRING TYPES only. 
2. String Constant and MapFrom/To and regex pattern needs to be quoted

## MapFrom
This rule dictates the pattern to match with given string for extraction. Some of the major components of the MapFrom syntax are followings.
- A literal: Matches literal characters / strings
- wildcard: `%*` matches any numbers of bytes, and `%?` matches any 1 byte.
- Variable: `%%varName%%` variable name surrounded by double `%` characters. The name can be referred later in `-mapc`'s MapTo string.
> Variable can have some additional attributes, and the syntax for variables looks like below.<br>
```%%VarName[:@class][:[chars]][:min[-max]][,brks]%%```
    * `:@class`: specifies the types of characters to match, such as `n` for 0~9, or `a` for a-z, and `b` for A-Z. 
    * `[chars]`: specifies which characters to be included in the variable. 
    * `min-max`: integers to restrict min and max numbers of characters 
    * `,brks`: specifies the list of characters at which the extraction should stop. 
    
    E.g. `%%CAPITAL:@b:3-5%%`: should match a substring of capital alphabetical letter, length between 3 ~ 5.<br>
    More attributes for MapFrom are available at [RT MapFrom Syntax](http://auriq.com/documentation/source/reference/manpages/aq_pp.html#rt-mapfrom-syntax)<br>
    **Note:** Each attributes are separated by `;`
    
    
## MapTo

This string specifies the format to map the extracted value into destination column. It has somewhat similar rule to MapFrom syntax, such as
- Literal: map constant characters / string
- Variable: place the value stored in named variable. This variable can also take some attributes, looks like following.
> ```%%VarName[:cnv][[:start]:length][,brks]%%```<br>
    * `:cnv`: either `b64` or `url[Num]` convert the variable string to base 64 or url accordingly.
    * `:start:length`: starting bytes position and subsequent length of the extracted variable. without the starting position, numbers of bytes from the beginning of the extracted variable.
    * `,brks`: list of characters to stop substitution of variable.
    
    **Note:** each attributes are separated by `;`<br>
    Details of MapTo is also available at [MapTo Syntax](http://auriq.com/documentation/source/reference/manpages/aq_pp.html#mapto-syntax)
    
## -Map(c/f)

### MapFrom

Let's start with simple string extraction using the options and variables.

**Extracting Origin of Flight**<br>

Using `-mapf` to extract the origin, and `-mapf` to map it to newly created column, here are the arguments to think about.

- `-mapf`
    - `ColName`: column to extract data from. This should be `search` column.
    - `MapFrom`: pattern to match the 3 capital letter airport code following "From=" string, followed by "To=...". We'll be using a variable named `ORG` to extract and store the match.
- `-mapc`
    - `ColSpec`: column spec for new string column. We'll name it as Origin
    - `MapTo`: We can place variable name `ORG` to the destination column, since we're not formatting the result in any way.
    
Let's take a look 

In [8]:
# set up the column spec and file name
file="data/aq_pp/airline_sample.csv"
col="S:search"

aq_pp -f,+1 $file -d $col -mapf search 'From=%%ORG%%To=%*' -mapc S:Origin '%%ORG%%'

"search","Origin"
"From=HNDTo=NGODate=20150506Class=Y","HND"
"From=NGik0To=OKADate=20150425Class=1","NGik0"
"From=OKATo=NGODate=20l5o4A5Class=Y","OKA"
"From=OKATo=NGODate=20150425Class=S","OKA"
"From=OKATo=nG0ODate=20150425Class=3","OKA"
"From=3GOTo=OKADate=S0150F25Class=Y","3GO"
"From=NGOTo=OKADate=20150419Class=Y","NGO"
"From=OKaTo=NGODate=20150517Class=Y","OKa"
"From=OKATo=NGODate=20150517Class=Y","OKA"
"From=OKATo=NGODate=20150517Class=Y","OKA"
"From=OKATo=NGODate=20150418Class=Y","OKA"
"From=HNDTo=NGODate=20150815Class=Y","HND"
"From=HNDTo=NGODate=20150815Class=Y","HND"
"From=NGOTo=HNDDate=20150815Class=Y","NGO"
"From=NGOTo=HNDDate=20150815Class=Y","NGO"
"From=SDJTo=NGODate=20150830Class=Y","SDJ"
"From=NGOTo=SDJDate=20150828Class=Y","NGO"
"From=HNDTo=OKADate=20150820Class=Y","HND"
"From=HNDTo=OKADate=20150820Class=Y","HND"
"From=SDJTo=NGODate=20150410Class=Y","SDJ"
"From=SDJTo=NGODate=20150410Class=Y","SDJ"
"From=SDJTo=NGODate=20151206Class=Y","SDJ"
"From=NGOTo=SDJDate=20151205Cla

"From=NGOTo=SDJDate=20150925Class=Y","NGO"
"From=NGOTo=SDJDate=20150925Class=Y","NGO"
"From=NGOTo=SDJDate=20150926Class=Y","NGO"
"From=NGOTo=SDJDate=20150927Class=Y","NGO"
"From=SDJTo=NGODate=20150927Class=Y","SDJ"
"From=SDJTo=NGODate=20150927Class=Y","SDJ"
"From=NGOTo=SDJDate=20150927Class=Y","NGO"
"From=NGOTo=SDJDate=20150926Class=Y","NGO"
"From=NGOTo=SDJDate=20150925Class=Y","NGO"
"From=NGOTo=SDJDate=20150926Class=Y","NGO"
"From=NGOTo=SDJDate=20150926Class=Y","NGO"
"From=NGOTo=SDJDate=20150927Class=Y","NGO"
"From=SDJTo=NGODate=20150927Class=Y","SDJ"
"From=SDJTo=NGODate=20150926Class=Y","SDJ"
"From=NGOTo=SDJDate=20150926Class=Y","NGO"
"From=NGOTo=SDJDate=20150926Class=Y","NGO"
"From=SDJTo=NGODate=20150926Class=Y","SDJ"
"From=SDJTo=NGODate=20150927Class=Y","SDJ"
"From=NGOTo=SDJDate=20150927Class=Y","NGO"
"From=NGOTo=SDJDate=20150926Class=Y","NGO"
"From=NGOTo=SDJDate=20150925Class=Y","NGO"
"From=NGOTo=SDJDate=20150926Class=Y","NGO"
"From=NGOTo=SDJDate=20150924Class=Y","NGO"
"From=NGOTo

"From=SPKTo=OKADate=20160521Class=Y","SPK"
"From=SPKTo=OKADate=20160521Class=S","SPK"
"From=SPKTo=OKADate=20160521Class=Y","SPK"
"From=OKATo=SPKDate=20160521Class=Y","OKA"
"From=OKATo=SPKDate=20160521Class=S","OKA"
"From=OKATo=SPKDate=20160521Class=Y","OKA"
"From=OKATo=SPKDate=20160521Class=S","OKA"
"From=OKATo=SPKDate=20160521Class=S","OKA"
"From=OKATo=SPKDate=20160521Class=S","OKA"
"From=OKATo=SPKDate=20160521Class=Y","OKA"
"From=OKATo=SPKDate=20160521Class=S","OKA"
"From=OKATo=SPKDate=20160521Class=Y","OKA"
"From=SPKTo=OKADate=20160521Class=Y","SPK"
"From=SPKTo=OKADate=20160521Class=S","SPK"
"From=SPKTo=OKADate=20160521Class=Y","SPK"
"From=NGOTo=OKADate=20160518Class=Y","NGO"
"From=NGOTo=OKADate=20160518Class=Y","NGO"
"From=NGOTo=OKADate=20160518Class=Y","NGO"
"From=NGOTo=OKADate=20160518Class=S","NGO"
"From=NGOTo=OKADate=20160518Class=Y","NGO"
"From=ISGTo=NGODate=20160528Class=Y","ISG"
"From=NGOTo=ISGDate=20160528Class=Y","NGO"
"From=ISGTo=NGODate=20160528Class=Y","ISG"
"From=NGOTo

"From=NGOTo=SDJDate=20160522Class=Y","NGO"
"From=NGOTo=HNDDate=20160713Class=Y","NGO"
"From=ISGTo=NGODate=20161025Class=Y","ISG"
"From=ISGTo=NGODate=20161025Class=Y","ISG"
"From=ISGTo=NGODate=20161025Class=Y","ISG"
"From=ISGTo=NGODate=20161025Class=Y","ISG"
"From=ISGTo=NGODate=20161025Class=Y","ISG"
"From=ISGTo=NGODate=20161025Class=Y","ISG"
"From=ISGTo=NGODate=20161025Class=Y","ISG"
"From=HNDTo=SPKDate=20160919Class=Y","HND"
"From=HNDTo=SPKDate=20160919Class=Y","HND"
"From=HNDTo=SPKDate=20160919Class=Y","HND"
"From=HNDTo=SPKDate=20160919Class=S","HND"
"From=HNDTo=SPKDate=20160919Class=S","HND"
"From=HNDTo=SPKDate=20160919Class=S","HND"
"From=HNDTo=SPKDate=20160919Class=Y","HND"
"From=HNDTo=SPKDate=20161009Class=Y","HND"
"From=SPKTo=HNDDate=20161009Class=Y","SPK"
"From=SPKTo=HNDDate=20161009Class=S","SPK"
"From=SPKTo=HNDDate=20161009Class=Y","SPK"
"From=SPKTo=HNDDate=20161010Class=Y","SPK"
"From=SPKTo=HNDDate=20161010Class=Y","SPK"
"From=SPKTo=HNDDate=20161010Class=Y","SPK"
"From=SPKTo

"From=HNDTo=KMQDate=20180427Class=Y","HND"
"From=HNDTo=KMQDate=20180427Class=Y","HND"
"From=HNDTo=KMQDate=20180420Class=Y","HND"
"From=HNDTo=KMQDate=20180420Class=Y","HND"
"From=HNDTo=NGODate=20180704Class=Y","HND"
"From=HNDTo=NGODate=20180705Class=Y","HND"
"From=HNDTo=NGODate=20180705Class=Y","HND"
"From=NGOTo=SDJDate=20181006Class=Y","NGO"
"From=HNDTo=KMQDate=20180919Class=Y","HND"
"From=KMQTo=HNDDate=20180919Class=Y","KMQ"
"From=KMQTo=HNDDate=20180920Class=Y","KMQ"
"From=KMQTo=HNDDate=20180921Class=Y","KMQ"
"From=HNDTo=KMQDate=20180927Class=Y","HND"
"From=HNDTo=KMQDate=20180928Class=Y","HND"
"From=HNDTo=KMQDate=20180928Class=Y","HND"
"From=HNDTo=KMQDate=20181204Class=Y","HND"
"From=HNDTo=KMQDate=20181204Class=Y","HND"
"From=HNDTo=KMQDate=20181205Class=Y","HND"
"From=HNDTo=KMQDate=20181205Class=Y","HND"
"From=HNDTo=KMQDate=20181205Class=Y","HND"
"From=HNDTo=KMQDate=20181205Class=Y","HND"
"From=HNDTo=KMQDate=20181204Class=Y","HND"
"From=KMQTo=HNDDate=20181207Class=Y","KMQ"
"From=KMQTo

...

"From=FUKTo=ITMDate=20151212Class=Y","FUK"
"From=ITMTo=FUKDate=20160119Class=Y","ITM"
"From=ITMTo=FUKDate=20160120Class=Y","ITM"
"From=FUKTo=ITMDate=20151014Class=Y","FUK"
"From=FUKTo=ITMDate=20151212Class=Y","FUK"
"From=FUKTo=ITMDate=20151212Class=Y","FUK"
"From=FUKTo=ITMDate=20151212Class=Y","FUK"
"From=ITMTo=HNDDate=20151119Class=Y","ITM"
"From=ITMTo=HNDDate=20151119Class=Y","ITM"
"From=ITMTo=HNDDate=20151119Class=Y","ITM"
"From=ITMTo=HNDDate=20151119Class=Y","ITM"
"From=ITMTo=HNDDate=20151119Class=Y","ITM"
"From=FUKTo=ITMDate=20160120Class=Y","FUK"
"From=FUKTo=ITMDate=20151225Class=Y","FUK"
"From=HNDTo=KOJDate=20160423Class=Y","HND"
"From=HNDTo=KOJDate=20160424Class=Y","HND"
"From=ITMTo=FUKDate=20160513Class=Y","ITM"
"From=FUKTo=ITMDate=20160425Class=Y","FUK"
"From=ITMTo=FUKDate=20160704Class=Y","ITM"
"From=FUKTo=ITMDate=20160705Class=Y","FUK"
"From=ITMTo=FUKDate=20160715Class=Y","ITM"
"From=ITMTo=KMJDate=20160522Class=Y","ITM"
"From=KIXTo=FUKDate=20160522Class=Y","KIX"
"From=KIXTo

"From=ITMTo=HNDDate=20151029Class=Y","ITM"
"From=OSATo=HNDDate=20151029Class=Y","OSA"
"From=HNDTo=OSADate=20151029Class=Y","HND"
"From=HNDTo=OSADate=20151030Class=Y","HND"
"From=FUKTo=ITMDate=20150910Class=Y","FUK"
"From=FUKTo=ITMDate=20151014Class=Y","FUK"
"From=ITMTo=FUKDate=20151105Class=Y","ITM"
"From=ITMTo=KMJDate=20151231Class=Y","ITM"
"From=HNDTo=ITMDate=20151124Class=Y","HND"
"From=HNDTo=OSADate=20151124Class=Y","HND"
"From=ITMTo=KMJDate=20151231Class=Y","ITM"
"From=KMJTo=ITMDate=20151231Class=Y","KMJ"
"From=KMJTo=ITMDate=20160101Class=Y","KMJ"
"From=KMJTo=ITMDate=20160102Class=Y","KMJ"
"From=KMJTo=ITMDate=20160103Class=Y","KMJ"
"From=HNDTo=ITMDate=20151120Class=Y","HND"
"From=ITMTo=HNDDate=20151120Class=Y","ITM"
"From=OSATo=HNDDate=20151120Class=Y","OSA"
"From=ITMTo=FUKDate=20151121Class=Y","ITM"
"From=FUKTo=ITMDate=20151121Class=Y","FUK"
"From=HNDTo=ITMDate=20151030Class=Y","HND"
"From=HNDTo=FUKDate=20151120Class=Y","HND"
"From=ITMTo=HNDDate=20151029Class=Y","ITM"
"From=HNDTo

"From=HNDTo=ITMDate=20160823Class=Y","HND"
"From=HNDTo=OSADate=20160823Class=Y","HND"
"From=ITMTo=FUKDate=20160917Class=Y","ITM"
"From=ITMTo=FUKDate=20160916Class=Y","ITM"
"From=FUKTo=ITMDate=20161001Class=Y","FUK"
"From=FUKTo=ITMDate=20160916Class=Y","FUK"
"From=ITMTo=FUKDate=20160917Class=Y","ITM"
"From=ITMTo=HNDDate=20161006Class=Y","ITM"
"From=HNDTo=UKBDate=20161007Class=Y","HND"
"From=ITMTo=HNDDate=20161012Class=Y","ITM"
"From=ITMTo=HNDDate=20161012Class=Y","ITM"
"From=OSATo=HNDDate=20161012Class=Y","OSA"
"From=OSATo=HNDDate=20161012Class=Y","OSA"
"From=OSATo=HNDDate=20161012Class=Y","OSA"
"From=HNDTo=UKBDate=20161012Class=Y","HND"
"From=ITMTo=HNDDate=20161012Class=Y","ITM"
"From=HNDTo=UKBDate=20161012Class=Y","HND"
"From=ITMTo=HNDDate=20161012Class=Y","ITM"
"From=ITMTo=HNDDate=20161011Class=Y","ITM"
"From=ITMTo=FUKDate=20161219Class=Y","ITM"
"From=ITMTo=HNDDate=20161026Class=Y","ITM"
"From=HNDTo=ITMDate=20161026Class=Y","HND"
"From=HNDTo=OSADate=20161026Class=Y","HND"
"From=HNDTo

"From=OKATo=ISGDate=20150406Class=Y","OKA"
"From=OKATo=ISGDate=20150405Class=Y","OKA"
"From=ISGTo=OKADate=20150428Class=Y","ISG"
"From=OKATo=ISGDate=20150428Class=Y","OKA"
"From=OKATo=ISGDate=20150429Class=Y","OKA"
"From=ISGTo=OKADate=20150403Class=Y","ISG"
"From=ISGTo=OKADate=20150403Class=Y","ISG"
"From=ISGTo=OKADate=20150403Class=Y","ISG"
"From=ISGTo=OKADate=20150403Class=Y","ISG"
"From=ISGTo=HNDDate=20150904Class=Y","ISG"
"From=ISGTo=HNDDate=20150904Class=Y","ISG"
"From=ISGTo=OKADate=20150904Class=Y","ISG"
"From=ISGTo=HNDDate=20150904Class=Y","ISG"
"From=ISGTo=OKADate=20150904Class=Y","ISG"
"From=ISGTo=HNDDate=20150904Class=Y","ISG"
"From=ISGTo=HNDDate=20150904Class=Y","ISG"
"From=ISGTo=HNDDate=20150904Class=Y","ISG"
"From=ISGTo=OKADate=20150904Class=Y","ISG"
"From=ISGTo=HNDDate=20150904Class=Y","ISG"
"From=ISGTo=OKADate=20150904Class=Y","ISG"
"From=OKATo=HNDDate=20150904Class=Y","OKA"
"From=OKATo=ISGDate=20150405Class=Y","OKA"
"From=OKATo=ISGDate=20150406Class=Y","OKA"
"From=OKATo

"From=OKATo=ISGDate=20150405Class=Y","OKA"
"From=OKATo=ISGDate=20150404Class=Y","OKA"
"From=ISGTo=OKADate=20150404Class=Y","ISG"
"From=ISGTo=OKADate=20150403Class=Y","ISG"
"From=ISGTo=OKADate=20150404Class=Y","ISG"
"From=ISGTo=OKADate=20150406Class=Y","ISG"
"From=ISGTo=OKADate=20150405Class=Y","ISG"
"From=ISGTo=OKADate=20150404Class=Y","ISG"
"From=ISGTo=OKADate=20150825Class=Y","ISG"
"From=ISGTo=OKADate=20150918Class=Y","ISG"
"From=ISGTo=OKADate=20150919Class=Y","ISG"
"From=ISGTo=OKADate=20150920Class=Y","ISG"
"From=ISGTo=OKADate=20150919Class=Y","ISG"
"From=OKATo=ISGDate=20150924Class=Y","OKA"
"From=OKATo=ISGDate=20150820Class=Y","OKA"
"From=OKATo=ISGDate=20150821Class=Y","OKA"
"From=OKATo=ISGDate=20150822Class=Y","OKA"
"From=OKATo=ISGDate=20150823Class=Y","OKA"
"From=ISGTo=OKADate=20150823Class=Y","ISG"
"From=ISGTo=OKADate=20150824Class=Y","ISG"
"From=ISGTo=OKADate=20150825Class=Y","ISG"
"From=ISGTo=OKADate=20150821Class=Y","ISG"
"From=ISGTo=OKADate=20150820Class=Y","ISG"
"From=ISGTo

Takig a closer look at MapFrom string, we have 
```From=%%ORG%%To=%*```<br>
- `From=`: literal to match the substring before the origin airport code
- `%%ORG%%`: variable named ORG
- `To=`: String literal
- `%*`: wildcard to match everything after "To=" in the column

Precisely speaking, this expression. matches **ANY** substring between `From=` and `To=` which is followed by any numbers of bytes, and stores it in variable named `ORG`.

**Specifying Format of Variable - string**<br>

What if there are anomalies in the dataset, which include invalid airport code in the position between `From=` and `To=`? 
With the MapFrom expression above, it'll still pick up the invalid origin code and map it to the new column. In fact, take a look at the result above. You can see that some invalid airport codes are extracted as well, such as `NGik0`.

How can we make sure that we're only extracting valid airport code? 
We can do it by specifying format of variable with attributes.

In this case, we want to make sure that the extracted airport code look like...
- capital alphabetical characters
- exactly 3 letters long

This can be achieved by <br>
```%%ORG:@b:3-3%%```<br>
where
- `@b`: class to specify the characters are in range of A~Z
- `3-3`: min and max numbers of characters in variable

In [9]:
aq_pp -f,+1 $file -d $col -mapf search 'From=%%ORG:@b:3-3%%To=%*' -mapc S:Origin '%%ORG%%'

"search","Origin"
"From=HNDTo=NGODate=20150506Class=Y","HND"
"From=NGik0To=OKADate=20150425Class=1",
"From=OKATo=NGODate=20l5o4A5Class=Y","OKA"
"From=OKATo=NGODate=20150425Class=S","OKA"
"From=OKATo=nG0ODate=20150425Class=3","OKA"
"From=3GOTo=OKADate=S0150F25Class=Y",
"From=NGOTo=OKADate=20150419Class=Y","NGO"
"From=OKaTo=NGODate=20150517Class=Y",
"From=OKATo=NGODate=20150517Class=Y","OKA"
"From=OKATo=NGODate=20150517Class=Y","OKA"
"From=OKATo=NGODate=20150418Class=Y","OKA"
"From=HNDTo=NGODate=20150815Class=Y","HND"
"From=HNDTo=NGODate=20150815Class=Y","HND"
"From=NGOTo=HNDDate=20150815Class=Y","NGO"
"From=NGOTo=HNDDate=20150815Class=Y","NGO"
"From=SDJTo=NGODate=20150830Class=Y","SDJ"
"From=NGOTo=SDJDate=20150828Class=Y","NGO"
"From=HNDTo=OKADate=20150820Class=Y","HND"
"From=HNDTo=OKADate=20150820Class=Y","HND"
"From=SDJTo=NGODate=20150410Class=Y","SDJ"
"From=SDJTo=NGODate=20150410Class=Y","SDJ"
"From=SDJTo=NGODate=20151206Class=Y","SDJ"
"From=NGOTo=SDJDate=20151205Class=Y","NGO"
"From

"From=NGOTo=SDJDate=20150925Class=Y","NGO"
"From=NGOTo=SDJDate=20150926Class=Y","NGO"
"From=NGOTo=SDJDate=20150927Class=Y","NGO"
"From=SDJTo=NGODate=20150927Class=Y","SDJ"
"From=SDJTo=NGODate=20150927Class=Y","SDJ"
"From=NGOTo=SDJDate=20150927Class=Y","NGO"
"From=NGOTo=SDJDate=20150926Class=Y","NGO"
"From=NGOTo=SDJDate=20150925Class=Y","NGO"
"From=NGOTo=SDJDate=20150926Class=Y","NGO"
"From=NGOTo=SDJDate=20150926Class=Y","NGO"
"From=NGOTo=SDJDate=20150927Class=Y","NGO"
"From=SDJTo=NGODate=20150927Class=Y","SDJ"
"From=SDJTo=NGODate=20150926Class=Y","SDJ"
"From=NGOTo=SDJDate=20150926Class=Y","NGO"
"From=NGOTo=SDJDate=20150926Class=Y","NGO"
"From=SDJTo=NGODate=20150926Class=Y","SDJ"
"From=SDJTo=NGODate=20150927Class=Y","SDJ"
"From=NGOTo=SDJDate=20150927Class=Y","NGO"
"From=NGOTo=SDJDate=20150926Class=Y","NGO"
"From=NGOTo=SDJDate=20150925Class=Y","NGO"
"From=NGOTo=SDJDate=20150926Class=Y","NGO"
"From=NGOTo=SDJDate=20150924Class=Y","NGO"
"From=NGOTo=HNDDate=20151008Class=Y","NGO"
"From=NGOTo

"From=SPKTo=OKADate=20160521Class=S","SPK"
"From=SPKTo=OKADate=20160521Class=Y","SPK"
"From=OKATo=SPKDate=20160521Class=Y","OKA"
"From=OKATo=SPKDate=20160521Class=S","OKA"
"From=OKATo=SPKDate=20160521Class=Y","OKA"
"From=OKATo=SPKDate=20160521Class=S","OKA"
"From=OKATo=SPKDate=20160521Class=S","OKA"
"From=OKATo=SPKDate=20160521Class=S","OKA"
"From=OKATo=SPKDate=20160521Class=Y","OKA"
"From=OKATo=SPKDate=20160521Class=S","OKA"
"From=OKATo=SPKDate=20160521Class=Y","OKA"
"From=SPKTo=OKADate=20160521Class=Y","SPK"
"From=SPKTo=OKADate=20160521Class=S","SPK"
"From=SPKTo=OKADate=20160521Class=Y","SPK"
"From=NGOTo=OKADate=20160518Class=Y","NGO"
"From=NGOTo=OKADate=20160518Class=Y","NGO"
"From=NGOTo=OKADate=20160518Class=Y","NGO"
"From=NGOTo=OKADate=20160518Class=S","NGO"
"From=NGOTo=OKADate=20160518Class=Y","NGO"
"From=ISGTo=NGODate=20160528Class=Y","ISG"
"From=NGOTo=ISGDate=20160528Class=Y","NGO"
"From=ISGTo=NGODate=20160528Class=Y","ISG"
"From=NGOTo=ISGDate=20160528Class=Y","NGO"
"From=ISGTo

"From=NGOTo=HNDDate=20160713Class=Y","NGO"
"From=ISGTo=NGODate=20161025Class=Y","ISG"
"From=ISGTo=NGODate=20161025Class=Y","ISG"
"From=ISGTo=NGODate=20161025Class=Y","ISG"
"From=ISGTo=NGODate=20161025Class=Y","ISG"
"From=ISGTo=NGODate=20161025Class=Y","ISG"
"From=ISGTo=NGODate=20161025Class=Y","ISG"
"From=ISGTo=NGODate=20161025Class=Y","ISG"
"From=HNDTo=SPKDate=20160919Class=Y","HND"
"From=HNDTo=SPKDate=20160919Class=Y","HND"
"From=HNDTo=SPKDate=20160919Class=Y","HND"
"From=HNDTo=SPKDate=20160919Class=S","HND"
"From=HNDTo=SPKDate=20160919Class=S","HND"
"From=HNDTo=SPKDate=20160919Class=S","HND"
"From=HNDTo=SPKDate=20160919Class=Y","HND"
"From=HNDTo=SPKDate=20161009Class=Y","HND"
"From=SPKTo=HNDDate=20161009Class=Y","SPK"
"From=SPKTo=HNDDate=20161009Class=S","SPK"
"From=SPKTo=HNDDate=20161009Class=Y","SPK"
"From=SPKTo=HNDDate=20161010Class=Y","SPK"
"From=SPKTo=HNDDate=20161010Class=Y","SPK"
"From=SPKTo=HNDDate=20161010Class=Y","SPK"
"From=SPKTo=HNDDate=20161009Class=Y","SPK"
"From=HNDTo

"From=HNDTo=KMQDate=20180427Class=Y","HND"
"From=HNDTo=KMQDate=20180420Class=Y","HND"
"From=HNDTo=KMQDate=20180420Class=Y","HND"
"From=HNDTo=NGODate=20180704Class=Y","HND"
"From=HNDTo=NGODate=20180705Class=Y","HND"
"From=HNDTo=NGODate=20180705Class=Y","HND"
"From=NGOTo=SDJDate=20181006Class=Y","NGO"
"From=HNDTo=KMQDate=20180919Class=Y","HND"
"From=KMQTo=HNDDate=20180919Class=Y","KMQ"
"From=KMQTo=HNDDate=20180920Class=Y","KMQ"
"From=KMQTo=HNDDate=20180921Class=Y","KMQ"
"From=HNDTo=KMQDate=20180927Class=Y","HND"
"From=HNDTo=KMQDate=20180928Class=Y","HND"
"From=HNDTo=KMQDate=20180928Class=Y","HND"
"From=HNDTo=KMQDate=20181204Class=Y","HND"
"From=HNDTo=KMQDate=20181204Class=Y","HND"
"From=HNDTo=KMQDate=20181205Class=Y","HND"
"From=HNDTo=KMQDate=20181205Class=Y","HND"
"From=HNDTo=KMQDate=20181205Class=Y","HND"
"From=HNDTo=KMQDate=20181205Class=Y","HND"
"From=HNDTo=KMQDate=20181204Class=Y","HND"
"From=KMQTo=HNDDate=20181207Class=Y","KMQ"
"From=KMQTo=HNDDate=20181207Class=Y","KMQ"
"From=HNDTo

...

"From=SDJTo=ITMDate=20170928Class=S","SDJ"
"From=ITMTo=HNDDate=20171018Class=Y","ITM"
"From=HNDTo=FUKDate=20171125Class=Y","HND"
"From=HNDTo=FUKDate=20171125Class=Y","HND"
"From=HNDTo=FUKDate=20171125Class=Y","HND"
"From=HNDTo=KOJDate=20171125Class=Y","HND"
"From=HNDTo=KOJDate=20171125Class=S","HND"
"From=FUKTo=SDJDate=20171127Class=Y","FUK"
"From=FUKTo=SDJDate=20171127Class=Y","FUK"
"From=FUKTo=SDJDate=20171127Class=Y","FUK"
"From=HNDTo=ITMDate=20171016Class=Y","HND"
"From=HNDTo=ITMDate=20171016Class=S","HND"
"From=HNDTo=ITMDate=20171016Class=Y","HND"
"From=HNDTo=ITMDate=20171016Class=S","HND"
"From=SPKTo=HNDDate=20171031Class=Y","SPK"
"From=SPKTo=HNDDate=20171031Class=S","SPK"
"From=FUKTo=SDJDate=20171127Class=Y","FUK"
"From=FUKTo=SDJDate=20171127Class=Y","FUK"
"From=HNDTo=KOJDate=20171124Class=Y","HND"
"From=HNDTo=KOJDate=20171124Class=S","HND"
"From=HNDTo=SPKDate=20171028Class=S","HND"
"From=SPKTo=HNDDate=20171031Class=Y","SPK"
"From=SPKTo=HNDDate=20171031Class=S","SPK"
"From=SPKTo

"From=OSATo=HNDDate=20151224Class=Y","OSA"
"From=HNDTo=ITMDate=20151225Class=Y","HND"
"From=OSATo=NGSDate=20151204Class=Y","OSA"
"From=OSATo=KMJDate=20151217Class=Y","OSA"
"From=OSATo=KMJDate=20151216Class=Y","OSA"
"From=OSATo=FUKDate=20160114Class=Y","OSA"
"From=OSATo=HNDDate=20160202Class=Y","OSA"
"From=OKATo=NGSDate=20160120Class=Y","OKA"
"From=NKYTo=HNDDate=20160216Class=Y","NKY"
"From=OSATo=HNDDate=20160125Class=Y","OSA"
"From=NGSTo=OSADate=20160205Class=Y","NGS"
"From=OSATo=NKYDate=20160206Class=Y","OSA"
"From=OSATo=HNDDate=20160302Class=Y","OSA"
"From=OSATo=OKADate=20160324Class=Y","OSA"
"From=OSATo=OKADate=20160323Class=Y","OSA"
"From=OSATo=OKADate=20160428Class=Y","OSA"
"From=HNDTo=OKADate=20160428Class=Y","HND"
"From=OSATo=SPKDate=20160707Class=Y","OSA"
"From=FUKTo=OKADate=20160524Class=Y","FUK"
"From=OKATo=FUKDate=20160524Class=Y","OKA"
"From=OSATo=OKADate=20160524Class=Y","OSA"
"From=HNDTo=OSADate=20160518Class=Y","HND"
"From=HNDTo=OSADate=20160518Class=S","HND"
"From=HNDTo

"From=OSATo=HNDDate=20171228Class=S","OSA"
"From=HNDTo=KOJDate=20171226Class=Y","HND"
"From=HNDTo=KOJDate=20171226Class=S","HND"
"From=HNDTo=KOJDate=20171226Class=S","HND"
"From=KOJTo=ITMDate=20171227Class=S","KOJ"
"From=KOJTo=ITMDate=20171227Class=S","KOJ"
"From=ITMTo=HNDDate=20171228Class=Y","ITM"
"From=ITMTo=HNDDate=20171228Class=S","ITM"
"From=KOJTo=ITMDate=20171226Class=Y","KOJ"
"From=HNDTo=ITMDate=20171226Class=S","HND"
"From=HNDTo=ITMDate=20171226Class=S","HND"
"From=HNDTo=AXTDate=20180303Class=Y","HND"
"From=ITMTo=HNDDate=20180124Class=Y","ITM"
"From=SDJTo=ITMDate=20180122Class=Y","SDJ"
"From=SPKTo=OKADate=20180305Class=Y","SPK"
"From=SPKTo=OKADate=20180305Class=Y","SPK"
"From=SPKTo=OKADate=20180305Class=Y","SPK"
"From=SPKTo=OKADate=20180305Class=Y","SPK"
"From=ITMTo=HNDDate=20180124Class=Y","ITM"
"From=AXTTo=SPKDate=20180304Class=Y","AXT"
"From=AXTTo=SPKDate=20180304Class=Y","AXT"
"From=SPKTo=OKADate=20180305Class=Y","SPK"
"From=SPKTo=OKADate=20180305Class=Y","SPK"
"From=SPKTo

"From=FUKTo=HNDDate=20181104Class=Y","FUK"
"From=FUKTo=HNDDate=20181104Class=Y","FUK"
"From=HNDTo=ITMDate=20181207Class=S","HND"
"From=FUKTo=HNDDate=20181104Class=Y","FUK"
"From=FUKTo=HNDDate=20181104Class=S","FUK"
"From=FUKTo=HNDDate=20181104Class=Y","FUK"
"From=HNDTo=HKDDate=20190101Class=Y","HND"
"From=HNDTo=HKDDate=20190101Class=Y","HND"
"From=HNDTo=SPKDate=20190101Class=Y","HND"
"From=FUKTo=HNDDate=20181104Class=S","FUK"
"From=FUKTo=HNDDate=20181104Class=S","FUK"
"From=FUKTo=HNDDate=20181104Class=S","FUK"
"From=HNDTo=HKDDate=20181231Class=Y","HND"
"From=HNDTo=HKDDate=20190101Class=Y","HND"
"From=HNDTo=HKDDate=20190101Class=Y","HND"
"From=SPKTo=HKDDate=20181231Class=Y","SPK"
"From=SPKTo=HKDDate=20190101Class=Y","SPK"
"From=HNDTo=HKDDate=20181201Class=Y","HND"
"From=HNDTo=HKDDate=20190101Class=Y","HND"
"From=FUKTo=HNDDate=20181104Class=Y","FUK"
"From=FUKTo=HNDDate=20181104Class=S","FUK"
"From=SPKTo=HKDDate=20190102Class=Y","SPK"
"From=HKDTo=SPKDate=20190102Class=Y","HKD"
"From=HNDTo

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



As a result, only valid airport codes are extracted and mapped to Origin column while invalid ones are left blank (meaning no match was found).
This can be used as data validation method.

**Specifying Format of Variable - number**<br>

Let us try to extract numeric values, namely year, month and date data. We will be using similar tactics used above to filter out the invalid numeric record as well. Here are example of valid and invalid numeric record.

* Valid: `...Date=20150425Class...`
* Invalid: `...Date=S0150F25Class...`

Valid date data is followed by `..Date=` substring, and have a format of 8 digits nubmers, 4 digit for year, 2 for month and date.
Let's extract all 8 digits, store it in `TIME` variable, and map it in `time` column.

In [10]:
aq_pp -f,+1 $file -d $col -mapf search '%*Date=%%TIME:@n:8-8%%Class%*' -mapc S:time '%%TIME%%'

"search","time"
"From=HNDTo=NGODate=20150506Class=Y","20150506"
"From=NGik0To=OKADate=20150425Class=1","20150425"
"From=OKATo=NGODate=20l5o4A5Class=Y",
"From=OKATo=NGODate=20150425Class=S","20150425"
"From=OKATo=nG0ODate=20150425Class=3","20150425"
"From=3GOTo=OKADate=S0150F25Class=Y",
"From=NGOTo=OKADate=20150419Class=Y","20150419"
"From=OKaTo=NGODate=20150517Class=Y","20150517"
"From=OKATo=NGODate=20150517Class=Y","20150517"
"From=OKATo=NGODate=20150517Class=Y","20150517"
"From=OKATo=NGODate=20150418Class=Y","20150418"
"From=HNDTo=NGODate=20150815Class=Y","20150815"
"From=HNDTo=NGODate=20150815Class=Y","20150815"
"From=NGOTo=HNDDate=20150815Class=Y","20150815"
"From=NGOTo=HNDDate=20150815Class=Y","20150815"
"From=SDJTo=NGODate=20150830Class=Y","20150830"
"From=NGOTo=SDJDate=20150828Class=Y","20150828"
"From=HNDTo=OKADate=20150820Class=Y","20150820"
"From=HNDTo=OKADate=20150820Class=Y","20150820"
"From=SDJTo=NGODate=20150410Class=Y","20150410"
"From=SDJTo=NGODate=20150410Class=Y","201

"From=SDJTo=NGODate=20150927Class=Y","20150927"
"From=SDJTo=NGODate=20150927Class=Y","20150927"
"From=NGOTo=SDJDate=20150927Class=Y","20150927"
"From=NGOTo=SDJDate=20150926Class=Y","20150926"
"From=NGOTo=SDJDate=20150926Class=Y","20150926"
"From=SDJTo=NGODate=20150830Class=Y","20150830"
"From=SDJTo=NGODate=20151025Class=Y","20151025"
"From=SDJTo=NGODate=20151025Class=Y","20151025"
"From=SDJTo=NGODate=20151025Class=Y","20151025"
"From=HNDTo=SPKDate=20160206Class=Y","20160206"
"From=HNDTo=SPKDate=20160206Class=Y","20160206"
"From=SDJTo=NGODate=20150927Class=Y","20150927"
"From=SDJTo=NGODate=20150927Class=Y","20150927"
"From=NGOTo=SDJDate=20150926Class=Y","20150926"
"From=NGOTo=SDJDate=20150926Class=Y","20150926"
"From=NGOTo=SDJDate=20150925Class=Y","20150925"
"From=NGOTo=SDJDate=20150925Class=Y","20150925"
"From=NGOTo=SPKDate=20160527Class=Y","20160527"
"From=NGOTo=SPKDate=20160528Class=Y","20160528"
"From=NGOTo=SDJDate=20150925Class=Y","20150925"
"From=NGOTo=SDJDate=20150925Class=Y","20

"From=NGOTo=OKADate=20160507Class=Y","20160507"
"From=NGOTo=ISGDate=20160505Class=Y","20160505"
"From=NGOTo=ISGDate=20160506Class=Y","20160506"
"From=ISGTo=NGODate=20160506Class=Y","20160506"
"From=ISGTo=NGODate=20160507Class=Y","20160507"
"From=NGOTo=ISGDate=20160507Class=Y","20160507"
"From=ISGTo=NGODate=20160507Class=Y","20160507"
"From=NGOTo=ISGDate=20160507Class=Y","20160507"
"From=ISGTo=NGODate=20160507Class=Y","20160507"
"From=ISGTo=NGODate=20160506Class=Y","20160506"
"From=NGOTo=ISGDate=20160506Class=Y","20160506"
"From=NGOTo=ISGDate=20160505Class=Y","20160505"
"From=HNDTo=ISGDate=20160617Class=Y","20160617"
"From=HNDTo=ISGDate=20160505Class=Y","20160505"
"From=HNDTo=ISGDate=20160505Class=S","20160505"
"From=ISGTo=HNDDate=20160505Class=S","20160505"
"From=ISGTo=HNDDate=20160506Class=S","20160506"
"From=ISGTo=HNDDate=20160507Class=S","20160507"
"From=ISGTo=HNDDate=20160507Class=S","20160507"
"From=ISGTo=HNDDate=20160507Class=S","20160507"
"From=ISGTo=HNDDate=20160507Class=Y","20

"From=OKATo=NGODate=20160529Class=S","20160529"
"From=NGOTo=OKADate=20160529Class=S","20160529"
"From=NGOTo=OKADate=20160528Class=S","20160528"
"From=NGOTo=OKADate=20160528Class=Y","20160528"
"From=NGOTo=SPKDate=20160528Class=Y","20160528"
"From=NGOTo=SPKDate=20160528Class=Y","20160528"
"From=NGOTo=SPKDate=20160528Class=Y","20160528"
"From=NGOTo=SPKDate=20160528Class=S","20160528"
"From=NGOTo=SPKDate=20160528Class=S","20160528"
"From=NGOTo=SPKDate=20160528Class=S","20160528"
"From=NGOTo=SPKDate=20160528Class=Y","20160528"
"From=NGOTo=MMYDate=20160528Class=Y","20160528"
"From=NGOTo=ISGDate=20160528Class=Y","20160528"
"From=NGOTo=ISGDate=20160528Class=Y","20160528"
"From=NGOTo=ISGDate=20160528Class=S","20160528"
"From=NGOTo=ISGDate=20160528Class=S","20160528"
"From=NGOTo=ISGDate=20160528Class=S","20160528"
"From=NGOTo=ISGDate=20160529Class=S","20160529"
"From=ISGTo=NGODate=20160529Class=S","20160529"
"From=ISGTo=NGODate=20160529Class=Y","20160529"
"From=ISGTo=NGODate=20160529Class=Y","20

"From=NGOTo=SDJDate=20170124Class=Y","20170124"
"From=NGOTo=SDJDate=20170124Class=Y","20170124"
"From=SDJTo=NGODate=20170301Class=Y","20170301"
"From=HNDTo=SPKDate=20170716Class=Y","20170716"
"From=HNDTo=SPKDate=20170716Class=Y","20170716"
"From=HNDTo=SPKDate=20170717Class=Y","20170717"
"From=SPKTo=HNDDate=20170717Class=Y","20170717"
"From=SPKTo=HNDDate=20170718Class=Y","20170718"
"From=SPKTo=HNDDate=20170718Class=Y","20170718"
"From=SPKTo=HNDDate=20170718Class=Y","20170718"
"From=NGOTo=SDJDate=20170424Class=Y","20170424"
"From=NGOTo=SDJDate=20170424Class=Y","20170424"
"From=NGOTo=SDJDate=20170424Class=Y","20170424"
"From=NGOTo=SDJDate=20170425Class=Y","20170425"
"From=NGOTo=SDJDate=20170516Class=Y","20170516"
"From=NGOTo=SDJDate=20170523Class=Y","20170523"
"From=NGOTo=SDJDate=20170516Class=Y","20170516"
"From=NGOTo=SDJDate=20170425Class=Y","20170425"
"From=NGOTo=SDJDate=20170728Class=Y","20170728"
"From=HNDTo=SPKDate=20170927Class=Y","20170927"
"From=HNDTo=SPKDate=20170924Class=Y","20

...

"From=HNDTo=ITMDate=20180413Class=Y","20180413"
"From=HNDTo=KIXDate=20180413Class=Y","20180413"
"From=HNDTo=KIXDate=20180413Class=S","20180413"
"From=HNDTo=FUKDate=20180422Class=Y","20180422"
"From=HNDTo=FUKDate=20180422Class=Y","20180422"
"From=HNDTo=FUKDate=20180422Class=Y","20180422"
"From=HNDTo=FUKDate=20180422Class=Y","20180422"
"From=HNDTo=FUKDate=20180422Class=Y","20180422"
"From=HNDTo=FUKDate=20180422Class=Y","20180422"
"From=HNDTo=FUKDate=20180422Class=Y","20180422"
"From=HNDTo=FUKDate=20180422Class=Y","20180422"
"From=HNDTo=FUKDate=20180422Class=Y","20180422"
"From=FUKTo=HNDDate=20180428Class=S","20180428"
"From=HNDTo=FUKDate=20180422Class=S","20180422"
"From=HNDTo=FUKDate=20180422Class=S","20180422"
"From=FUKTo=HNDDate=20180428Class=Y","20180428"
"From=HNDTo=FUKDate=20180422Class=Y","20180422"
"From=HNDTo=FUKDate=20180422Class=S","20180422"
"From=HNDTo=FUKDate=20180422Class=S","20180422"
"From=HNDTo=FUKDate=20180422Class=Y","20180422"
"From=HNDTo=KOJDate=20180405Class=Y","20

"From=HNDTo=KMQDate=20190113Class=Y","20190113"
"From=HNDTo=KMQDate=20190113Class=Y","20190113"
"From=HNDTo=SPKDate=20190101Class=S","20190101"
"From=HNDTo=SPKDate=20190101Class=S","20190101"
"From=KIXTo=HNDDate=20181209Class=Y","20181209"
"From=ITMTo=HNDDate=20181209Class=Y","20181209"
"From=HNDTo=ITMDate=20181209Class=Y","20181209"
"From=HNDTo=SPKDate=20190101Class=S","20190101"
"From=HNDTo=SPKDate=20190101Class=S","20190101"
"From=ITMTo=HNDDate=20181209Class=Y","20181209"
"From=ITMTo=HNDDate=20181209Class=S","20181209"
"From=HNDTo=SPKDate=20190101Class=Y","20190101"
"From=HNDTo=SPKDate=20190101Class=S","20190101"
"From=HNDTo=KMQDate=20190120Class=Y","20190120"
"From=HNDTo=SPKDate=20190101Class=Y","20190101"
"From=HNDTo=SPKDate=20190101Class=Y","20190101"
"From=ITMTo=HNDDate=20181209Class=Y","20181209"
"From=ITMTo=HNDDate=20181209Class=Y","20181209"
"From=ITMTo=HNDDate=20181209Class=Y","20181209"
"From=ITMTo=HNDDate=20181209Class=Y","20181209"
"From=HNDTo=SPKDate=20190101Class=S","20

"From=SPKTo=HNDDate=20190103Class=Y","20190103"
"From=HKDTo=HNDDate=20190103Class=Y","20190103"
"From=HNDTo=KMQDate=20190113Class=Y","20190113"
"From=KMQTo=HNDDate=20190114Class=Y","20190114"
"From=HNDTo=OSADate=20181207Class=Y","20181207"
"From=HNDTo=OSADate=20181207Class=S","20181207"
"From=HNDTo=OSADate=20181207Class=Y","20181207"
"From=OSATo=HNDDate=20181209Class=Y","20181209"
"From=KMQTo=HNDDate=20190114Class=Y","20190114"
"From=HNDTo=SPKDate=20190101Class=Y","20190101"
"From=HNDTo=SPKDate=20190101Class=S","20190101"
"From=HNDTo=SPKDate=20190101Class=Y","20190101"
"From=HNDTo=SPKDate=20190101Class=Y","20190101"
"From=HNDTo=SPKDate=20190101Class=S","20190101"
"From=HNDTo=SPKDate=20190101Class=Y","20190101"
"From=HNDTo=SPKDate=20190101Class=S","20190101"
"From=HNDTo=NTQDate=20190113Class=Y","20190113"
"From=HNDTo=KMQDate=20190113Class=Y","20190113"
"From=HNDTo=KMQDate=20190114Class=Y","20190114"
"From=KMQTo=HNDDate=20190114Class=Y","20190114"
"From=HNDTo=KMQDate=20190113Class=Y","20

"From=FUKTo=ITMDate=20151014Class=Y","20151014"
"From=FUKTo=ITMDate=20151212Class=Y","20151212"
"From=FUKTo=ITMDate=20151212Class=Y","20151212"
"From=FUKTo=ITMDate=20151212Class=Y","20151212"
"From=ITMTo=HNDDate=20151119Class=Y","20151119"


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Little about MapFrom string, ```%*Date=%%TIME:@n:8-8%%Class%*```<br>
- `%*`: 2 of them are used in the beginning and at the end to skip the unneccesary string.
- `Date=`, `Class`: literals that preceed and follows the variable
- `%%TIME:@n:8-8%%`:variable, with attributes of ...
    - `@n`: numbers of 0~9
    - `8-8`: min 8 and max 8 length
    
You can see that only valid date data are extracted, and rest are left blank.

### MapTo

We've focused on usage of MapFrom with `-mapf/c` to extract patterns, and kept use of MapTo simple in previous sections.<br>
In this part of the notebook, we'll take a look at how to use MapTo string to specify formatting of destination column.

In this example, we'll keep using the 8 digits date string extracted above, and add MapTo string. <br>

**Literal and Variable**<br>
To make the content of the column clear, we'd like to map the date digits in format of <br>
```date=20180904```.

How can we map this using MapTo? We knows that `%%varName%%` will substitute the extracted value. All we need is the literal string of `date=` before that. Let's take a look.

In [12]:
aq_pp -f,+1 $file -d $col -mapf search '%*Date=%%TIME:@n:8-8%%Class%*' \
-mapc S:time 'date=%%TIME%%'

"search","time"
"From=HNDTo=NGODate=20150506Class=Y","date=20150506"
"From=NGik0To=OKADate=20150425Class=1","date=20150425"
"From=OKATo=NGODate=20l5o4A5Class=Y","date="
"From=OKATo=NGODate=20150425Class=S","date=20150425"
"From=OKATo=nG0ODate=20150425Class=3","date=20150425"
"From=3GOTo=OKADate=S0150F25Class=Y","date="
"From=NGOTo=OKADate=20150419Class=Y","date=20150419"
"From=OKaTo=NGODate=20150517Class=Y","date=20150517"
"From=OKATo=NGODate=20150517Class=Y","date=20150517"
"From=OKATo=NGODate=20150517Class=Y","date=20150517"
"From=OKATo=NGODate=20150418Class=Y","date=20150418"
"From=HNDTo=NGODate=20150815Class=Y","date=20150815"
"From=HNDTo=NGODate=20150815Class=Y","date=20150815"
"From=NGOTo=HNDDate=20150815Class=Y","date=20150815"
"From=NGOTo=HNDDate=20150815Class=Y","date=20150815"
"From=SDJTo=NGODate=20150830Class=Y","date=20150830"
"From=NGOTo=SDJDate=20150828Class=Y","date=20150828"
"From=HNDTo=OKADate=20150820Class=Y","date=20150820"
"From=HNDTo=OKADate=20150820Class=Y","date=

"From=SPKTo=HNDDate=20160207Class=Y","date=20160207"
"From=SPKTo=HNDDate=20160207Class=Y","date=20160207"
"From=SPKTo=HNDDate=20160207Class=Y","date=20160207"
"From=SPKTo=HNDDate=20160207Class=Y","date=20160207"
"From=SPKTo=HNDDate=20160207Class=S","date=20160207"
"From=SPKTo=HNDDate=20160207Class=S","date=20160207"
"From=SPKTo=HNDDate=20160207Class=S","date=20160207"
"From=SPKTo=HNDDate=20160207Class=S","date=20160207"
"From=NGOTo=SDJDate=20150828Class=Y","date=20150828"
"From=NGOTo=SDJDate=20150828Class=Y","date=20150828"
"From=SDJTo=NGODate=20150830Class=Y","date=20150830"
"From=SDJTo=NGODate=20150830Class=Y","date=20150830"
"From=NGOTo=SDJDate=20150926Class=Y","date=20150926"
"From=NGOTo=SDJDate=20150926Class=Y","date=20150926"
"From=NGOTo=SDJDate=20150927Class=Y","date=20150927"
"From=SDJTo=NGODate=20150927Class=Y","date=20150927"
"From=SDJTo=NGODate=20150927Class=Y","date=20150927"
"From=SDJTo=NGODate=20150927Class=Y","date=20150927"
"From=NGOTo=SDJDate=20150927Class=Y","date=201

"From=HNDTo=ISGDate=20160416Class=Y","date=20160416"
"From=HNDTo=ISGDate=20160423Class=Y","date=20160423"
"From=HNDTo=ISGDate=20160423Class=Y","date=20160423"
"From=HNDTo=ISGDate=20160423Class=Y","date=20160423"
"From=HNDTo=ISGDate=20160423Class=Y","date=20160423"
"From=HNDTo=ISGDate=20160423Class=Y","date=20160423"
"From=HNDTo=ISGDate=20160423Class=Y","date=20160423"
"From=HNDTo=ISGDate=20160423Class=Y","date=20160423"
"From=NGOTo=ISGDate=20160507Class=Y","date=20160507"
"From=NGOTo=ISGDate=20160507Class=S","date=20160507"
"From=NGOTo=ISGDate=20160507Class=S","date=20160507"
"From=NGOTo=ISGDate=20160507Class=S","date=20160507"
"From=NGOTo=ISGDate=20160507Class=Y","date=20160507"
"From=NGOTo=SPKDate=20160527Class=Y","date=20160527"
"From=NGOTo=SPKDate=20160528Class=Y","date=20160528"
"From=NGOTo=SPKDate=20160529Class=Y","date=20160529"
"From=SPKTo=NGODate=20160529Class=Y","date=20160529"
"From=SPKTo=NGODate=20160529Class=Y","date=20160529"
"From=SPKTo=NGODate=20160529Class=Y","date=201

"From=HNDTo=SPKDate=20160504Class=Y","date=20160504"
"From=HNDTo=SPKDate=20160503Class=Y","date=20160503"
"From=HNDTo=SPKDate=20160429Class=Y","date=20160429"
"From=HNDTo=OKADate=20160521Class=Y","date=20160521"
"From=HNDTo=OKADate=20160522Class=Y","date=20160522"
"From=OKATo=HNDDate=20160522Class=Y","date=20160522"
"From=HNDTo=OKADate=20160522Class=Y","date=20160522"
"From=HNDTo=OKADate=20160521Class=Y","date=20160521"
"From=NGOTo=OKADate=20160507Class=Y","date=20160507"
"From=NGOTo=OKADate=20160507Class=Y","date=20160507"
"From=NGOTo=OKADate=20160507Class=S","date=20160507"
"From=NGOTo=OKADate=20160507Class=Y","date=20160507"
"From=NGOTo=OKADate=20160507Class=Y","date=20160507"
"From=NGOTo=OKADate=20160507Class=Y","date=20160507"
"From=NGOTo=OKADate=20160508Class=Y","date=20160508"
"From=OKATo=NGODate=20160508Class=Y","date=20160508"
"From=NGOTo=OKADate=20160508Class=Y","date=20160508"
"From=NGOTo=OKADate=20160507Class=Y","date=20160507"
"From=NGOTo=ISGDate=20160528Class=Y","date=201

"From=SPKTo=HNDDate=20161123Class=Y","date=20161123"
"From=SPKTo=HNDDate=20161123Class=S","date=20161123"
"From=SPKTo=HNDDate=20161123Class=Y","date=20161123"
"From=SPKTo=HNDDate=20161123Class=S","date=20161123"
"From=SPKTo=HNDDate=20161123Class=Y","date=20161123"
"From=SPKTo=HNDDate=20161123Class=Y","date=20161123"
"From=SPKTo=HNDDate=20161123Class=S","date=20161123"
"From=NGOTo=SPKDate=20161122Class=Y","date=20161122"
"From=NGOTo=SPKDate=20161122Class=Y","date=20161122"
"From=ISGTo=NGODate=20161026Class=Y","date=20161026"
"From=NRTTo=OKADate=20161205Class=Y","date=20161205"
"From=NRTTo=OKADate=20161205Class=Y","date=20161205"
"From=NRTTo=OKADate=20161205Class=Y","date=20161205"
"From=NRTTo=OKADate=20161206Class=Y","date=20161206"
"From=NRTTo=OKADate=20161205Class=Y","date=20161205"
"From=NRTTo=SPKDate=20161205Class=Y","date=20161205"
"From=NRTTo=SDJDate=20161223Class=Y","date=20161223"
"From=NRTTo=SDJDate=20161222Class=Y","date=20161222"
"From=NRTTo=SDJDate=20170108Class=Y","date=201

...

"From=OSATo=SPKDate=20180123Class=Y","date=20180123"
"From=SPKTo=HNDDate=20180124Class=Y","date=20180124"
"From=HNDTo=SPKDate=20171231Class=Y","date=20171231"
"From=HNDTo=SPKDate=20180101Class=Y","date=20180101"
"From=SPKTo=HNDDate=20180102Class=Y","date=20180102"
"From=SPKTo=HNDDate=20180103Class=Y","date=20180103"
"From=SPKTo=HNDDate=20180103Class=S","date=20180103"
"From=SPKTo=HNDDate=20180102Class=S","date=20180102"
"From=OSATo=HNDDate=20180123Class=Y","date=20180123"
"From=OSATo=HNDDate=20180124Class=Y","date=20180124"
"From=SDJTo=OSADate=20180122Class=Y","date=20180122"
"From=OSATo=SPKDate=20180123Class=Y","date=20180123"
"From=OSATo=SPKDate=20180123Class=Y","date=20180123"
"From=AXTTo=HNDDate=20180305Class=Y","date=20180305"
"From=AXTTo=HNDDate=20180304Class=Y","date=20180304"
"From=AXTTo=SPKDate=20180304Class=Y","date=20180304"
"From=HNDTo=OSADate=20180301Class=Y","date=20180301"
"From=ITMTo=AXTDate=20180304Class=Y","date=20180304"
"From=AXTTo=HNDDate=20180305Class=Y","date=201

"From=KMQTo=HNDDate=20190114Class=Y","date=20190114"
"From=HNDTo=KMQDate=20190113Class=Y","date=20190113"
"From=HNDTo=SPKDate=20190101Class=Y","date=20190101"
"From=HNDTo=SPKDate=20190101Class=S","date=20190101"
"From=HNDTo=OSADate=20181207Class=Y","date=20181207"
"From=OSATo=HNDDate=20181209Class=Y","date=20181209"
"From=HNDTo=SPKDate=20190101Class=Y","date=20190101"
"From=HNDTo=SPKDate=20190101Class=Y","date=20190101"
"From=HNDTo=FUKDate=20150418Class=Y","date=20150418"
"From=HNDTo=FUKDate=20150418Class=Y","date=20150418"
"From=HNDTo=FUKDate=20150418Class=Y","date=20150418"
"From=HNDTo=FUKDate=20150418Class=Y","date=20150418"
"From=HNDTo=KIXDate=20150530Class=Y","date=20150530"
"From=HNDTo=ITMDate=20150530Class=Y","date=20150530"
"From=HNDTo=KIXDate=20150918Class=Y","date=20150918"
"From=HNDTo=KIXDate=20150918Class=Y","date=20150918"
"From=HNDTo=KIXDate=20150918Class=Y","date=20150918"
"From=KIXTo=HNDDate=20150918Class=Y","date=20150918"
"From=HNDTo=KIXDate=20150918Class=Y","date=201

"From=ITMTo=HNDDate=20151119Class=Y","date=20151119"
"From=ITMTo=HNDDate=20151119Class=Y","date=20151119"
"From=ITMTo=HNDDate=20151119Class=Y","date=20151119"
"From=FUKTo=ITMDate=20160120Class=Y","date=20160120"
"From=FUKTo=ITMDate=20151225Class=Y","date=20151225"
"From=HNDTo=KOJDate=20160423Class=Y","date=20160423"
"From=HNDTo=KOJDate=20160424Class=Y","date=20160424"
"From=ITMTo=FUKDate=20160513Class=Y","date=20160513"
"From=FUKTo=ITMDate=20160425Class=Y","date=20160425"
"From=ITMTo=FUKDate=20160704Class=Y","date=20160704"
"From=FUKTo=ITMDate=20160705Class=Y","date=20160705"
"From=ITMTo=FUKDate=20160715Class=Y","date=20160715"
"From=ITMTo=KMJDate=20160522Class=Y","date=20160522"
"From=KIXTo=FUKDate=20160522Class=Y","date=20160522"
"From=KIXTo=FUKDate=20160523Class=Y","date=20160523"
"From=ITMTo=FUKDate=20160522Class=Y","date=20160522"
"From=ITMTo=FUKDate=20160715Class=Y","date=20160715"
"From=FUKTo=ITMDate=20160716Class=Y","date=20160716"
"From=FUKTo=FKSDate=20160514Class=Y","date=201

"From=ITMTo=HNDDate=20151020Class=Y","date=20151020"
"From=ITMTo=HNDDate=20151021Class=Y","date=20151021"
"From=HNDTo=ITMDate=20151021Class=Y","date=20151021"
"From=HNDTo=OSADate=20151021Class=Y","date=20151021"
"From=ITMTo=KOJDate=20151023Class=Y","date=20151023"
"From=ITMTo=FUKDate=20150908Class=Y","date=20150908"
"From=ITMTo=FUKDate=20150909Class=Y","date=20150909"
"From=FUKTo=ITMDate=20150909Class=Y","date=20150909"
"From=HNDTo=FUKDate=20151120Class=Y","date=20151120"
"From=ITMTo=HNDDate=20150925Class=Y","date=20150925"
"From=FUKTo=ITMDate=20150910Class=Y","date=20150910"
"From=ITMTo=FUKDate=20151015Class=Y","date=20151015"
"From=ITMTo=FUKDate=20151014Class=Y","date=20151014"
"From=FUKTo=ITMDate=20151014Class=Y","date=20151014"
"From=FUKTo=ITMDate=20151015Class=Y","date=20151015"
"From=FUKTo=ITMDate=20151016Class=Y","date=20151016"
"From=ITMTo=FUKDate=20151016Class=Y","date=20151016"
"From=KOJTo=ITMDate=20151023Class=Y","date=20151023"
"From=ITMTo=SPKDate=20151020Class=Y","date=201

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



**Attributes**<br>

There are 3 attributes available and we'll cover 2 of them, 
- `:length`: numbers of bytes from the beginning of the value stored in the variable
- `:start:length`: starting byte position and **subsequent** length of the extracted value. Starting at 0.

We've extracted 8 digits, but would only like to use year portion (first 4 digits) of the data. We can accomplish this using `:length` attribute.

In [16]:
aq_pp -f,+1 $file -d $col -mapf search '%*Date=%%TIME:@n:8-8%%Class%*' \
-mapc S:time 'Year=%%TIME:4%%'

"search","time"
"From=HNDTo=NGODate=20150506Class=Y","Year=2015"
"From=NGik0To=OKADate=20150425Class=1","Year=2015"
"From=OKATo=NGODate=20l5o4A5Class=Y","Year="
"From=OKATo=NGODate=20150425Class=S","Year=2015"
"From=OKATo=nG0ODate=20150425Class=3","Year=2015"
"From=3GOTo=OKADate=S0150F25Class=Y","Year="
"From=NGOTo=OKADate=20150419Class=Y","Year=2015"
"From=OKaTo=NGODate=20150517Class=Y","Year=2015"
"From=OKATo=NGODate=20150517Class=Y","Year=2015"
"From=OKATo=NGODate=20150517Class=Y","Year=2015"
"From=OKATo=NGODate=20150418Class=Y","Year=2015"
"From=HNDTo=NGODate=20150815Class=Y","Year=2015"
"From=HNDTo=NGODate=20150815Class=Y","Year=2015"
"From=NGOTo=HNDDate=20150815Class=Y","Year=2015"
"From=NGOTo=HNDDate=20150815Class=Y","Year=2015"
"From=SDJTo=NGODate=20150830Class=Y","Year=2015"
"From=NGOTo=SDJDate=20150828Class=Y","Year=2015"
"From=HNDTo=OKADate=20150820Class=Y","Year=2015"
"From=HNDTo=OKADate=20150820Class=Y","Year=2015"
"From=SDJTo=NGODate=20150410Class=Y","Year=2015"
"From=SDJ

"From=NGOTo=SDJDate=20150926Class=Y","Year=2015"
"From=NGOTo=SDJDate=20150926Class=Y","Year=2015"
"From=NGOTo=SDJDate=20150927Class=Y","Year=2015"
"From=SDJTo=NGODate=20150927Class=Y","Year=2015"
"From=SDJTo=NGODate=20150927Class=Y","Year=2015"
"From=SDJTo=NGODate=20150927Class=Y","Year=2015"
"From=NGOTo=SDJDate=20150927Class=Y","Year=2015"
"From=NGOTo=SDJDate=20150926Class=Y","Year=2015"
"From=NGOTo=SDJDate=20150926Class=Y","Year=2015"
"From=SDJTo=NGODate=20150830Class=Y","Year=2015"
"From=SDJTo=NGODate=20151025Class=Y","Year=2015"
"From=SDJTo=NGODate=20151025Class=Y","Year=2015"
"From=SDJTo=NGODate=20151025Class=Y","Year=2015"
"From=HNDTo=SPKDate=20160206Class=Y","Year=2016"
"From=HNDTo=SPKDate=20160206Class=Y","Year=2016"
"From=SDJTo=NGODate=20150927Class=Y","Year=2015"
"From=SDJTo=NGODate=20150927Class=Y","Year=2015"
"From=NGOTo=SDJDate=20150926Class=Y","Year=2015"
"From=NGOTo=SDJDate=20150926Class=Y","Year=2015"
"From=NGOTo=SDJDate=20150925Class=Y","Year=2015"
"From=NGOTo=SDJDate=

"From=NGOTo=SPKDate=20160529Class=Y","Year=2016"
"From=NGOTo=SPKDate=20160528Class=Y","Year=2016"
"From=NGOTo=SPKDate=20160527Class=Y","Year=2016"
"From=NGOTo=OKADate=20160507Class=Y","Year=2016"
"From=NGOTo=OKADate=20160508Class=Y","Year=2016"
"From=OKATo=NGODate=20160508Class=Y","Year=2016"
"From=NGOTo=OKADate=20160508Class=Y","Year=2016"
"From=NGOTo=OKADate=20160507Class=Y","Year=2016"
"From=NGOTo=ISGDate=20160505Class=Y","Year=2016"
"From=NGOTo=ISGDate=20160506Class=Y","Year=2016"
"From=ISGTo=NGODate=20160506Class=Y","Year=2016"
"From=ISGTo=NGODate=20160507Class=Y","Year=2016"
"From=NGOTo=ISGDate=20160507Class=Y","Year=2016"
"From=ISGTo=NGODate=20160507Class=Y","Year=2016"
"From=NGOTo=ISGDate=20160507Class=Y","Year=2016"
"From=ISGTo=NGODate=20160507Class=Y","Year=2016"
"From=ISGTo=NGODate=20160506Class=Y","Year=2016"
"From=NGOTo=ISGDate=20160506Class=Y","Year=2016"
"From=NGOTo=ISGDate=20160505Class=Y","Year=2016"
"From=HNDTo=ISGDate=20160617Class=Y","Year=2016"
"From=HNDTo=ISGDate=

"From=NGOTo=OKADate=20160528Class=S","Year=2016"
"From=NGOTo=OKADate=20160528Class=S","Year=2016"
"From=NGOTo=OKADate=20160528Class=S","Year=2016"
"From=NGOTo=OKADate=20160529Class=S","Year=2016"
"From=OKATo=NGODate=20160529Class=S","Year=2016"
"From=OKATo=NGODate=20160529Class=S","Year=2016"
"From=OKATo=NGODate=20160529Class=S","Year=2016"
"From=OKATo=NGODate=20160529Class=Y","Year=2016"
"From=OKATo=NGODate=20160529Class=Y","Year=2016"
"From=OKATo=NGODate=20160529Class=Y","Year=2016"
"From=OKATo=NGODate=20160529Class=S","Year=2016"
"From=NGOTo=OKADate=20160529Class=S","Year=2016"
"From=NGOTo=OKADate=20160528Class=S","Year=2016"
"From=NGOTo=OKADate=20160528Class=Y","Year=2016"
"From=NGOTo=SPKDate=20160528Class=Y","Year=2016"
"From=NGOTo=SPKDate=20160528Class=Y","Year=2016"
"From=NGOTo=SPKDate=20160528Class=Y","Year=2016"
"From=NGOTo=SPKDate=20160528Class=S","Year=2016"
"From=NGOTo=SPKDate=20160528Class=S","Year=2016"
"From=NGOTo=SPKDate=20160528Class=S","Year=2016"
"From=NGOTo=SPKDate=

"From=SPKTo=NGODate=20170205Class=Y","Year=2017"
"From=SPKTo=NGODate=20170205Class=Y","Year=2017"
"From=SPKTo=NGODate=20170205Class=Y","Year=2017"
"From=HNDTo=SPKDate=20170204Class=Y","Year=2017"
"From=HNDTo=SPKDate=20170204Class=Y","Year=2017"
"From=HNDTo=SPKDate=20170204Class=Y","Year=2017"
"From=SPKTo=HNDDate=20170305Class=Y","Year=2017"
"From=SPKTo=HNDDate=20170205Class=Y","Year=2017"
"From=HNDTo=SPKDate=20170207Class=Y","Year=2017"
"From=HNDTo=NGODate=20161211Class=Y","Year=2016"
"From=NGOTo=HNDDate=20161212Class=Y","Year=2016"
"From=NGOTo=SDJDate=20170124Class=Y","Year=2017"
"From=NGOTo=SDJDate=20170124Class=Y","Year=2017"
"From=NGOTo=SDJDate=20170124Class=Y","Year=2017"
"From=NGOTo=SDJDate=20170124Class=Y","Year=2017"
"From=SDJTo=NGODate=20170301Class=Y","Year=2017"
"From=HNDTo=SPKDate=20170716Class=Y","Year=2017"
"From=HNDTo=SPKDate=20170716Class=Y","Year=2017"
"From=HNDTo=SPKDate=20170717Class=Y","Year=2017"
"From=SPKTo=HNDDate=20170717Class=Y","Year=2017"
"From=SPKTo=HNDDate=

...

"From=NGOTo=KOJDate=20160904Class=Y","Year=2016"
"From=OSATo=NGSDate=20160818Class=Y","Year=2016"
"From=FUKTo=HNDDate=20161015Class=Y","Year=2016"
"From=OSATo=KMJDate=20160913Class=Y","Year=2016"
"From=OSATo=SPKDate=20161118Class=Y","Year=2016"
"From=SPKTo=OSADate=20161120Class=Y","Year=2016"
"From=FUKTo=OSADate=20161007Class=Y","Year=2016"
"From=OSATo=OITDate=20161004Class=Y","Year=2016"
"From=FUKTo=HNDDate=20161015Class=Y","Year=2016"
"From=HNDTo=KIXDate=20161016Class=Y","Year=2016"
"From=HNDTo=KIXDate=20161017Class=Y","Year=2016"
"From=OSATo=FUKDate=20161005Class=Y","Year=2016"
"From=OSATo=OITDate=20161018Class=Y","Year=2016"
"From=OSATo=HNDDate=20161016Class=Y","Year=2016"
"From=FUKTo=HNDDate=20161015Class=Y","Year=2016"
"From=HNDTo=OSADate=20161016Class=Y","Year=2016"
"From=OSATo=OITDate=20161018Class=Y","Year=2016"
"From=OSATo=OITDate=20161019Class=Y","Year=2016"
"From=OSATo=OITDate=20161018Class=Y","Year=2016"
"From=FUKTo=HNDDate=20161015Class=Y","Year=2016"
"From=HNDTo=OSADate=

"From=SDJTo=ITMDate=20180122Class=Y","Year=2018"
"From=SPKTo=OKADate=20180305Class=Y","Year=2018"
"From=SPKTo=OKADate=20180305Class=Y","Year=2018"
"From=SPKTo=OKADate=20180305Class=Y","Year=2018"
"From=SPKTo=OKADate=20180305Class=Y","Year=2018"
"From=ITMTo=HNDDate=20180124Class=Y","Year=2018"
"From=AXTTo=SPKDate=20180304Class=Y","Year=2018"
"From=AXTTo=SPKDate=20180304Class=Y","Year=2018"
"From=SPKTo=OKADate=20180305Class=Y","Year=2018"
"From=SPKTo=OKADate=20180305Class=Y","Year=2018"
"From=SPKTo=OKADate=20180203Class=Y","Year=2018"
"From=SPKTo=OKADate=20180305Class=Y","Year=2018"
"From=SPKTo=OKADate=20180305Class=S","Year=2018"
"From=SPKTo=OKADate=20180203Class=Y","Year=2018"
"From=SPKTo=OKADate=20180305Class=Y","Year=2018"
"From=SPKTo=OKADate=20180305Class=Y","Year=2018"
"From=SPKTo=OKADate=20180305Class=Y","Year=2018"
"From=SPKTo=OKADate=20180305Class=Y","Year=2018"
"From=SPKTo=OKADate=20180305Class=Y","Year=2018"
"From=HNDTo=OKADate=20180305Class=S","Year=2018"
"From=HNDTo=AXTDate=

"From=FUKTo=HNDDate=20181104Class=S","Year=2018"
"From=FUKTo=HNDDate=20181104Class=S","Year=2018"
"From=FUKTo=HNDDate=20181104Class=S","Year=2018"
"From=HNDTo=SPKDate=20190101Class=Y","Year=2019"
"From=HNDTo=SPKDate=20190101Class=Y","Year=2019"
"From=HNDTo=SPKDate=20190101Class=Y","Year=2019"
"From=HNDTo=SPKDate=20190101Class=Y","Year=2019"
"From=HNDTo=ITMDate=20181207Class=Y","Year=2018"
"From=FUKTo=HNDDate=20181103Class=Y","Year=2018"
"From=FUKTo=HNDDate=20181103Class=Y","Year=2018"
"From=FUKTo=HNDDate=20181104Class=Y","Year=2018"
"From=FUKTo=HNDDate=20181104Class=Y","Year=2018"
"From=HNDTo=ITMDate=20181207Class=S","Year=2018"
"From=FUKTo=HNDDate=20181104Class=Y","Year=2018"
"From=FUKTo=HNDDate=20181104Class=S","Year=2018"
"From=FUKTo=HNDDate=20181104Class=Y","Year=2018"
"From=HNDTo=HKDDate=20190101Class=Y","Year=2019"
"From=HNDTo=HKDDate=20190101Class=Y","Year=2019"
"From=HNDTo=SPKDate=20190101Class=Y","Year=2019"
"From=FUKTo=HNDDate=20181104Class=S","Year=2018"
"From=FUKTo=HNDDate=

"From=HNDTo=OKADate=20180919Class=Y","Year=2018"
"From=HNDTo=OKADate=20180919Class=S","Year=2018"
"From=HNDTo=OKADate=20180919Class=Y","Year=2018"
"From=HNDTo=OKADate=20180919Class=Y","Year=2018"
"From=HNDTo=OKADate=20180919Class=S","Year=2018"
"From=HNDTo=OKADate=20180919Class=Y","Year=2018"
"From=HNDTo=OKADate=20180919Class=S","Year=2018"
"From=OKATo=HNDDate=20180926Class=S","Year=2018"
"From=HNDTo=FUKDate=20181031Class=Y","Year=2018"
"From=HNDTo=NKYDate=20181031Class=Y","Year=2018"
"From=FUKTo=HNDDate=20181104Class=Y","Year=2018"
"From=HNDTo=FUKDate=20181031Class=Y","Year=2018"
"From=HNDTo=FUKDate=20181031Class=Y","Year=2018"
"From=HNDTo=SPKDate=20181007Class=Y","Year=2018"
"From=HNDTo=SPKDate=20190101Class=Y","Year=2019"
"From=SPKTo=HNDDate=20190103Class=Y","Year=2019"
"From=SPKTo=HNDDate=20190102Class=Y","Year=2019"
"From=SPKTo=HNDDate=20190101Class=Y","Year=2019"
"From=SPKTo=HNDDate=20181231Class=Y","Year=2018"
"From=HKDTo=SPKDate=20190103Class=Y","Year=2019"
"From=HKDTo=HNDDate=

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



What if we'd like to map only month (2 digits in the middle)? We can use `:start:length` attributes. Note that `start` is starting byte to substitute, an index starting from zero, and `length` is subsequent length.<br>
Let's take a look.

In [19]:
aq_pp -f,+1 $file -d $col -mapf search '%*Date=%%TIME:@n:8-8%%Class%*' \
-mapc S:time 'Month=%%TIME:4:2%%'

"search","time"
"From=HNDTo=NGODate=20150506Class=Y","Month=05"
"From=NGik0To=OKADate=20150425Class=1","Month=04"
"From=OKATo=NGODate=20l5o4A5Class=Y","Month="
"From=OKATo=NGODate=20150425Class=S","Month=04"
"From=OKATo=nG0ODate=20150425Class=3","Month=04"
"From=3GOTo=OKADate=S0150F25Class=Y","Month="
"From=NGOTo=OKADate=20150419Class=Y","Month=04"
"From=OKaTo=NGODate=20150517Class=Y","Month=05"
"From=OKATo=NGODate=20150517Class=Y","Month=05"
"From=OKATo=NGODate=20150517Class=Y","Month=05"
"From=OKATo=NGODate=20150418Class=Y","Month=04"
"From=HNDTo=NGODate=20150815Class=Y","Month=08"
"From=HNDTo=NGODate=20150815Class=Y","Month=08"
"From=NGOTo=HNDDate=20150815Class=Y","Month=08"
"From=NGOTo=HNDDate=20150815Class=Y","Month=08"
"From=SDJTo=NGODate=20150830Class=Y","Month=08"
"From=NGOTo=SDJDate=20150828Class=Y","Month=08"
"From=HNDTo=OKADate=20150820Class=Y","Month=08"
"From=HNDTo=OKADate=20150820Class=Y","Month=08"
"From=SDJTo=NGODate=20150410Class=Y","Month=04"
"From=SDJTo=NGODate=20150

"From=SDJTo=NGODate=20150927Class=Y","Month=09"
"From=SDJTo=NGODate=20150927Class=Y","Month=09"
"From=NGOTo=SDJDate=20150927Class=Y","Month=09"
"From=NGOTo=SDJDate=20150926Class=Y","Month=09"
"From=NGOTo=SDJDate=20150926Class=Y","Month=09"
"From=SDJTo=NGODate=20150830Class=Y","Month=08"
"From=SDJTo=NGODate=20151025Class=Y","Month=10"
"From=SDJTo=NGODate=20151025Class=Y","Month=10"
"From=SDJTo=NGODate=20151025Class=Y","Month=10"
"From=HNDTo=SPKDate=20160206Class=Y","Month=02"
"From=HNDTo=SPKDate=20160206Class=Y","Month=02"
"From=SDJTo=NGODate=20150927Class=Y","Month=09"
"From=SDJTo=NGODate=20150927Class=Y","Month=09"
"From=NGOTo=SDJDate=20150926Class=Y","Month=09"
"From=NGOTo=SDJDate=20150926Class=Y","Month=09"
"From=NGOTo=SDJDate=20150925Class=Y","Month=09"
"From=NGOTo=SDJDate=20150925Class=Y","Month=09"
"From=NGOTo=SPKDate=20160527Class=Y","Month=05"
"From=NGOTo=SPKDate=20160528Class=Y","Month=05"
"From=NGOTo=SDJDate=20150925Class=Y","Month=09"
"From=NGOTo=SDJDate=20150925Class=Y","Mo

"From=NGOTo=OKADate=20160507Class=Y","Month=05"
"From=NGOTo=ISGDate=20160505Class=Y","Month=05"
"From=NGOTo=ISGDate=20160506Class=Y","Month=05"
"From=ISGTo=NGODate=20160506Class=Y","Month=05"
"From=ISGTo=NGODate=20160507Class=Y","Month=05"
"From=NGOTo=ISGDate=20160507Class=Y","Month=05"
"From=ISGTo=NGODate=20160507Class=Y","Month=05"
"From=NGOTo=ISGDate=20160507Class=Y","Month=05"
"From=ISGTo=NGODate=20160507Class=Y","Month=05"
"From=ISGTo=NGODate=20160506Class=Y","Month=05"
"From=NGOTo=ISGDate=20160506Class=Y","Month=05"
"From=NGOTo=ISGDate=20160505Class=Y","Month=05"
"From=HNDTo=ISGDate=20160617Class=Y","Month=06"
"From=HNDTo=ISGDate=20160505Class=Y","Month=05"
"From=HNDTo=ISGDate=20160505Class=S","Month=05"
"From=ISGTo=HNDDate=20160505Class=S","Month=05"
"From=ISGTo=HNDDate=20160506Class=S","Month=05"
"From=ISGTo=HNDDate=20160507Class=S","Month=05"
"From=ISGTo=HNDDate=20160507Class=S","Month=05"
"From=ISGTo=HNDDate=20160507Class=S","Month=05"
"From=ISGTo=HNDDate=20160507Class=Y","Mo

"From=OKATo=NGODate=20160529Class=S","Month=05"
"From=NGOTo=OKADate=20160529Class=S","Month=05"
"From=NGOTo=OKADate=20160528Class=S","Month=05"
"From=NGOTo=OKADate=20160528Class=Y","Month=05"
"From=NGOTo=SPKDate=20160528Class=Y","Month=05"
"From=NGOTo=SPKDate=20160528Class=Y","Month=05"
"From=NGOTo=SPKDate=20160528Class=Y","Month=05"
"From=NGOTo=SPKDate=20160528Class=S","Month=05"
"From=NGOTo=SPKDate=20160528Class=S","Month=05"
"From=NGOTo=SPKDate=20160528Class=S","Month=05"
"From=NGOTo=SPKDate=20160528Class=Y","Month=05"
"From=NGOTo=MMYDate=20160528Class=Y","Month=05"
"From=NGOTo=ISGDate=20160528Class=Y","Month=05"
"From=NGOTo=ISGDate=20160528Class=Y","Month=05"
"From=NGOTo=ISGDate=20160528Class=S","Month=05"
"From=NGOTo=ISGDate=20160528Class=S","Month=05"
"From=NGOTo=ISGDate=20160528Class=S","Month=05"
"From=NGOTo=ISGDate=20160529Class=S","Month=05"
"From=ISGTo=NGODate=20160529Class=S","Month=05"
"From=ISGTo=NGODate=20160529Class=Y","Month=05"
"From=ISGTo=NGODate=20160529Class=Y","Mo

"From=NGOTo=SDJDate=20170124Class=Y","Month=01"
"From=NGOTo=SDJDate=20170124Class=Y","Month=01"
"From=SDJTo=NGODate=20170301Class=Y","Month=03"
"From=HNDTo=SPKDate=20170716Class=Y","Month=07"
"From=HNDTo=SPKDate=20170716Class=Y","Month=07"
"From=HNDTo=SPKDate=20170717Class=Y","Month=07"
"From=SPKTo=HNDDate=20170717Class=Y","Month=07"
"From=SPKTo=HNDDate=20170718Class=Y","Month=07"
"From=SPKTo=HNDDate=20170718Class=Y","Month=07"
"From=SPKTo=HNDDate=20170718Class=Y","Month=07"
"From=NGOTo=SDJDate=20170424Class=Y","Month=04"
"From=NGOTo=SDJDate=20170424Class=Y","Month=04"
"From=NGOTo=SDJDate=20170424Class=Y","Month=04"
"From=NGOTo=SDJDate=20170425Class=Y","Month=04"
"From=NGOTo=SDJDate=20170516Class=Y","Month=05"
"From=NGOTo=SDJDate=20170523Class=Y","Month=05"
"From=NGOTo=SDJDate=20170516Class=Y","Month=05"
"From=NGOTo=SDJDate=20170425Class=Y","Month=04"
"From=NGOTo=SDJDate=20170728Class=Y","Month=07"
"From=HNDTo=SPKDate=20170927Class=Y","Month=09"
"From=HNDTo=SPKDate=20170924Class=Y","Mo

...

"From=HNDTo=ITMDate=20180413Class=Y","Month=04"
"From=HNDTo=KIXDate=20180413Class=Y","Month=04"
"From=HNDTo=KIXDate=20180413Class=S","Month=04"
"From=HNDTo=FUKDate=20180422Class=Y","Month=04"
"From=HNDTo=FUKDate=20180422Class=Y","Month=04"
"From=HNDTo=FUKDate=20180422Class=Y","Month=04"
"From=HNDTo=FUKDate=20180422Class=Y","Month=04"
"From=HNDTo=FUKDate=20180422Class=Y","Month=04"
"From=HNDTo=FUKDate=20180422Class=Y","Month=04"
"From=HNDTo=FUKDate=20180422Class=Y","Month=04"
"From=HNDTo=FUKDate=20180422Class=Y","Month=04"
"From=HNDTo=FUKDate=20180422Class=Y","Month=04"
"From=FUKTo=HNDDate=20180428Class=S","Month=04"
"From=HNDTo=FUKDate=20180422Class=S","Month=04"
"From=HNDTo=FUKDate=20180422Class=S","Month=04"
"From=FUKTo=HNDDate=20180428Class=Y","Month=04"
"From=HNDTo=FUKDate=20180422Class=Y","Month=04"
"From=HNDTo=FUKDate=20180422Class=S","Month=04"
"From=HNDTo=FUKDate=20180422Class=S","Month=04"
"From=HNDTo=FUKDate=20180422Class=Y","Month=04"
"From=HNDTo=KOJDate=20180405Class=Y","Mo

"From=HNDTo=KMQDate=20190113Class=Y","Month=01"
"From=HNDTo=KMQDate=20190113Class=Y","Month=01"
"From=HNDTo=SPKDate=20190101Class=S","Month=01"
"From=HNDTo=SPKDate=20190101Class=S","Month=01"
"From=KIXTo=HNDDate=20181209Class=Y","Month=12"
"From=ITMTo=HNDDate=20181209Class=Y","Month=12"
"From=HNDTo=ITMDate=20181209Class=Y","Month=12"
"From=HNDTo=SPKDate=20190101Class=S","Month=01"
"From=HNDTo=SPKDate=20190101Class=S","Month=01"
"From=ITMTo=HNDDate=20181209Class=Y","Month=12"
"From=ITMTo=HNDDate=20181209Class=S","Month=12"
"From=HNDTo=SPKDate=20190101Class=Y","Month=01"
"From=HNDTo=SPKDate=20190101Class=S","Month=01"
"From=HNDTo=KMQDate=20190120Class=Y","Month=01"
"From=HNDTo=SPKDate=20190101Class=Y","Month=01"
"From=HNDTo=SPKDate=20190101Class=Y","Month=01"
"From=ITMTo=HNDDate=20181209Class=Y","Month=12"
"From=ITMTo=HNDDate=20181209Class=Y","Month=12"
"From=ITMTo=HNDDate=20181209Class=Y","Month=12"
"From=ITMTo=HNDDate=20181209Class=Y","Month=12"
"From=HNDTo=SPKDate=20190101Class=S","Mo

"From=SPKTo=HNDDate=20190103Class=Y","Month=01"
"From=HKDTo=HNDDate=20190103Class=Y","Month=01"
"From=HNDTo=KMQDate=20190113Class=Y","Month=01"
"From=KMQTo=HNDDate=20190114Class=Y","Month=01"
"From=HNDTo=OSADate=20181207Class=Y","Month=12"
"From=HNDTo=OSADate=20181207Class=S","Month=12"
"From=HNDTo=OSADate=20181207Class=Y","Month=12"
"From=OSATo=HNDDate=20181209Class=Y","Month=12"
"From=KMQTo=HNDDate=20190114Class=Y","Month=01"
"From=HNDTo=SPKDate=20190101Class=Y","Month=01"
"From=HNDTo=SPKDate=20190101Class=S","Month=01"
"From=HNDTo=SPKDate=20190101Class=Y","Month=01"
"From=HNDTo=SPKDate=20190101Class=Y","Month=01"
"From=HNDTo=SPKDate=20190101Class=S","Month=01"
"From=HNDTo=SPKDate=20190101Class=Y","Month=01"
"From=HNDTo=SPKDate=20190101Class=S","Month=01"
"From=HNDTo=NTQDate=20190113Class=Y","Month=01"
"From=HNDTo=KMQDate=20190113Class=Y","Month=01"
"From=HNDTo=KMQDate=20190114Class=Y","Month=01"
"From=KMQTo=HNDDate=20190114Class=Y","Month=01"
"From=HNDTo=KMQDate=20190113Class=Y","Mo

"From=FUKTo=ITMDate=20151014Class=Y","Month=10"
"From=FUKTo=ITMDate=20151212Class=Y","Month=12"
"From=FUKTo=ITMDate=20151212Class=Y","Month=12"
"From=FUKTo=ITMDate=20151212Class=Y","Month=12"
"From=ITMTo=HNDDate=20151119Class=Y","Month=11"
"From=ITMTo=HNDDate=20151119Class=Y","Month=11"
"From=ITMTo=HNDDate=20151119Class=Y","Month=11"
"From=ITMTo=HNDDate=20151119Class=Y","Month=11"
"From=ITMTo=HNDDate=20151119Class=Y","Month=11"
"From=FUKTo=ITMDate=20160120Class=Y","Month=01"
"From=FUKTo=ITMDate=20151225Class=Y","Month=12"
"From=HNDTo=KOJDate=20160423Class=Y","Month=04"
"From=HNDTo=KOJDate=20160424Class=Y","Month=04"
"From=ITMTo=FUKDate=20160513Class=Y","Month=05"
"From=FUKTo=ITMDate=20160425Class=Y","Month=04"
"From=ITMTo=FUKDate=20160704Class=Y","Month=07"
"From=FUKTo=ITMDate=20160705Class=Y","Month=07"
"From=ITMTo=FUKDate=20160715Class=Y","Month=07"
"From=ITMTo=KMJDate=20160522Class=Y","Month=05"
"From=KIXTo=FUKDate=20160522Class=Y","Month=05"
"From=KIXTo=FUKDate=20160523Class=Y","Mo

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



That looks correct!

In the 2 examples above, we have mapped part of `TIME` variable one time only.
However, in case if you'd like to map different part of one variable, for instance mapping year and month separately, we can do that as well.<br>
This is possible by referring variable name multiple times in a MapTo string in `-mapc` option.

In [20]:
aq_pp -f,+1 $file -d $col -mapf search '%*Date=%%TIME:@n:8-8%%Class%*' \
-mapc S:time 'Year=%%TIME:4%%, Month=%%TIME:4:2%%'

"search","time"
"From=HNDTo=NGODate=20150506Class=Y","Year=2015, Month=05"
"From=NGik0To=OKADate=20150425Class=1","Year=2015, Month=04"
"From=OKATo=NGODate=20l5o4A5Class=Y","Year=, Month="
"From=OKATo=NGODate=20150425Class=S","Year=2015, Month=04"
"From=OKATo=nG0ODate=20150425Class=3","Year=2015, Month=04"
"From=3GOTo=OKADate=S0150F25Class=Y","Year=, Month="
"From=NGOTo=OKADate=20150419Class=Y","Year=2015, Month=04"
"From=OKaTo=NGODate=20150517Class=Y","Year=2015, Month=05"
"From=OKATo=NGODate=20150517Class=Y","Year=2015, Month=05"
"From=OKATo=NGODate=20150517Class=Y","Year=2015, Month=05"
"From=OKATo=NGODate=20150418Class=Y","Year=2015, Month=04"
"From=HNDTo=NGODate=20150815Class=Y","Year=2015, Month=08"
"From=HNDTo=NGODate=20150815Class=Y","Year=2015, Month=08"
"From=NGOTo=HNDDate=20150815Class=Y","Year=2015, Month=08"
"From=NGOTo=HNDDate=20150815Class=Y","Year=2015, Month=08"
"From=SDJTo=NGODate=20150830Class=Y","Year=2015, Month=08"
"From=NGOTo=SDJDate=20150828Class=Y","Year=2015, 

"From=SPKTo=NGODate=20151228Class=Y","Year=2015, Month=12"
"From=SPKTo=NGODate=20160206Class=Y","Year=2016, Month=02"
"From=SPKTo=NGODate=20160207Class=Y","Year=2016, Month=02"
"From=SPKTo=HNDDate=20160207Class=Y","Year=2016, Month=02"
"From=SPKTo=HNDDate=20160207Class=Y","Year=2016, Month=02"
"From=SPKTo=HNDDate=20160207Class=Y","Year=2016, Month=02"
"From=SPKTo=HNDDate=20160207Class=S","Year=2016, Month=02"
"From=SPKTo=HNDDate=20160207Class=S","Year=2016, Month=02"
"From=SPKTo=HNDDate=20160207Class=S","Year=2016, Month=02"
"From=SPKTo=HNDDate=20160207Class=S","Year=2016, Month=02"
"From=SPKTo=HNDDate=20160207Class=S","Year=2016, Month=02"
"From=SPKTo=HNDDate=20160207Class=Y","Year=2016, Month=02"
"From=SPKTo=HNDDate=20160207Class=Y","Year=2016, Month=02"
"From=SPKTo=HNDDate=20160207Class=Y","Year=2016, Month=02"
"From=SPKTo=HNDDate=20160207Class=S","Year=2016, Month=02"
"From=SPKTo=HNDDate=20160207Class=Y","Year=2016, Month=02"
"From=SPKTo=HNDDate=20160207Class=Y","Year=2016, Month=0

"From=HNDTo=NGODate=20160404Class=S","Year=2016, Month=04"
"From=HNDTo=NGODate=20160404Class=Y","Year=2016, Month=04"
"From=NGOTo=OKADate=20160423Class=Y","Year=2016, Month=04"
"From=NGOTo=OKADate=20160424Class=Y","Year=2016, Month=04"
"From=OKATo=NGODate=20160424Class=Y","Year=2016, Month=04"
"From=OKATo=NGODate=20160425Class=Y","Year=2016, Month=04"
"From=OKATo=NGODate=20160424Class=Y","Year=2016, Month=04"
"From=NGOTo=OKADate=20160424Class=Y","Year=2016, Month=04"
"From=NGOTo=OKADate=20160423Class=Y","Year=2016, Month=04"
"From=NGOTo=SPKDate=20160528Class=Y","Year=2016, Month=05"
"From=NGOTo=SPKDate=20160527Class=Y","Year=2016, Month=05"
"From=NGOTo=SPKDate=20160527Class=Y","Year=2016, Month=05"
"From=NGOTo=SPKDate=20160527Class=Y","Year=2016, Month=05"
"From=NGOTo=SPKDate=20160527Class=Y","Year=2016, Month=05"
"From=NGOTo=SPKDate=20160527Class=Y","Year=2016, Month=05"
"From=NGOTo=SPKDate=20160527Class=Y","Year=2016, Month=05"
"From=NGOTo=SPKDate=20160527Class=Y","Year=2016, Month=0

"From=OKATo=NGODate=20160528Class=S","Year=2016, Month=05"
"From=HNDTo=OKADate=20160429Class=Y","Year=2016, Month=04"
"From=OKATo=HNDDate=20160429Class=Y","Year=2016, Month=04"
"From=OKATo=HNDDate=20160430Class=Y","Year=2016, Month=04"
"From=OKATo=HNDDate=20160501Class=Y","Year=2016, Month=05"
"From=OKATo=HNDDate=20160502Class=Y","Year=2016, Month=05"
"From=OKATo=HNDDate=20160503Class=Y","Year=2016, Month=05"
"From=OKATo=HNDDate=20160504Class=Y","Year=2016, Month=05"
"From=OKATo=HNDDate=20160505Class=Y","Year=2016, Month=05"
"From=OKATo=HNDDate=20160506Class=Y","Year=2016, Month=05"
"From=OKATo=HNDDate=20160505Class=Y","Year=2016, Month=05"
"From=OKATo=HNDDate=20160504Class=Y","Year=2016, Month=05"
"From=OKATo=HNDDate=20160503Class=Y","Year=2016, Month=05"
"From=OKATo=HNDDate=20160502Class=Y","Year=2016, Month=05"
"From=OKATo=HNDDate=20160430Class=Y","Year=2016, Month=04"
"From=OKATo=HNDDate=20160429Class=Y","Year=2016, Month=04"
"From=HNDTo=OKADate=20160429Class=Y","Year=2016, Month=0

"From=NGOTo=OKADate=20160518Class=Y","Year=2016, Month=05"
"From=NGOTo=OKADate=20160518Class=S","Year=2016, Month=05"
"From=NGOTo=OKADate=20160518Class=S","Year=2016, Month=05"
"From=NGOTo=OKADate=20160518Class=S","Year=2016, Month=05"
"From=OKATo=NGODate=20160518Class=S","Year=2016, Month=05"
"From=OKATo=NGODate=20160519Class=S","Year=2016, Month=05"
"From=OKATo=NGODate=20160519Class=S","Year=2016, Month=05"
"From=OKATo=NGODate=20160519Class=S","Year=2016, Month=05"
"From=OKATo=NGODate=20160518Class=S","Year=2016, Month=05"
"From=HNDTo=NGODate=20160521Class=Y","Year=2016, Month=05"
"From=HNDTo=NGODate=20160521Class=Y","Year=2016, Month=05"
"From=HNDTo=NGODate=20160521Class=Y","Year=2016, Month=05"
"From=NGOTo=SDJDate=20160522Class=Y","Year=2016, Month=05"
"From=SDJTo=NGODate=20160522Class=Y","Year=2016, Month=05"
"From=SDJTo=NGODate=20160522Class=Y","Year=2016, Month=05"
"From=SDJTo=NGODate=20160522Class=Y","Year=2016, Month=05"
"From=NGOTo=SDJDate=20160522Class=Y","Year=2016, Month=0

...

"From=HNDTo=ITMDate=20180714Class=Y","Year=2018, Month=07"
"From=HNDTo=ITMDate=20180714Class=S","Year=2018, Month=07"
"From=HNDTo=ITMDate=20180714Class=S","Year=2018, Month=07"
"From=HNDTo=OSADate=20180716Class=Y","Year=2018, Month=07"
"From=HNDTo=ITMDate=20180716Class=Y","Year=2018, Month=07"
"From=HNDTo=ITMDate=20180714Class=Y","Year=2018, Month=07"
"From=HNDTo=ITMDate=20180714Class=S","Year=2018, Month=07"
"From=HNDTo=ITMDate=20180714Class=S","Year=2018, Month=07"
"From=HNDTo=SPKDate=20180810Class=Y","Year=2018, Month=08"
"From=HNDTo=ITMDate=20180714Class=Y","Year=2018, Month=07"
"From=HNDTo=ITMDate=20180714Class=Y","Year=2018, Month=07"
"From=HNDTo=ITMDate=20180714Class=Y","Year=2018, Month=07"
"From=OSATo=HNDDate=20180716Class=S","Year=2018, Month=07"
"From=ITMTo=HNDDate=20180716Class=S","Year=2018, Month=07"
"From=OSATo=HNDDate=20180716Class=S","Year=2018, Month=07"
"From=ITMTo=HNDDate=20180716Class=S","Year=2018, Month=07"
"From=HNDTo=SPKDate=20180810Class=S","Year=2018, Month=0

"From=HNDTo=SPKDate=20190101Class=S","Year=2019, Month=01"
"From=HNDTo=KMQDate=20190120Class=Y","Year=2019, Month=01"
"From=HNDTo=SPKDate=20190101Class=Y","Year=2019, Month=01"
"From=HNDTo=SPKDate=20190101Class=Y","Year=2019, Month=01"
"From=ITMTo=HNDDate=20181209Class=Y","Year=2018, Month=12"
"From=ITMTo=HNDDate=20181209Class=Y","Year=2018, Month=12"
"From=ITMTo=HNDDate=20181209Class=Y","Year=2018, Month=12"
"From=ITMTo=HNDDate=20181209Class=Y","Year=2018, Month=12"
"From=HNDTo=SPKDate=20190101Class=S","Year=2019, Month=01"
"From=HNDTo=SPKDate=20190101Class=S","Year=2019, Month=01"
"From=HNDTo=FUKDate=20171231Class=Y","Year=2017, Month=12"
"From=HNDTo=FUKDate=20171231Class=S","Year=2017, Month=12"
"From=HNDTo=KOJDate=20180101Class=Y","Year=2018, Month=01"
"From=SDJTo=OSADate=20180122Class=Y","Year=2018, Month=01"
"From=OSATo=HNDDate=20171228Class=Y","Year=2017, Month=12"
"From=OSATo=HNDDate=20171228Class=S","Year=2017, Month=12"
"From=HNDTo=KOJDate=20171226Class=Y","Year=2017, Month=1

"From=HNDTo=SHBDate=20190102Class=Y","Year=2019, Month=01"
"From=HNDTo=SHBDate=20190101Class=Y","Year=2019, Month=01"
"From=HNDTo=AKJDate=20190101Class=Y","Year=2019, Month=01"
"From=HNDTo=AKJDate=20190102Class=Y","Year=2019, Month=01"
"From=HNDTo=SPKDate=20190101Class=Y","Year=2019, Month=01"
"From=HNDTo=SPKDate=20181231Class=Y","Year=2018, Month=12"
"From=HNDTo=SPKDate=20190101Class=Y","Year=2019, Month=01"
"From=SPKTo=HNDDate=20190103Class=Y","Year=2019, Month=01"
"From=SPKTo=HNDDate=20190104Class=Y","Year=2019, Month=01"
"From=SPKTo=HNDDate=20190105Class=Y","Year=2019, Month=01"
"From=HNDTo=OSADate=20181207Class=Y","Year=2018, Month=12"
"From=OSATo=HNDDate=20181209Class=Y","Year=2018, Month=12"
"From=HKDTo=SPKDate=20190103Class=Y","Year=2019, Month=01"
"From=HKDTo=HNDDate=20190103Class=Y","Year=2019, Month=01"
"From=HKDTo=HNDDate=20190104Class=Y","Year=2019, Month=01"
"From=HKDTo=HNDDate=20190103Class=Y","Year=2019, Month=01"
"From=HKDTo=HNDDate=20190102Class=Y","Year=2019, Month=0

"From=HNDTo=FUKDate=20150706Class=Y","Year=2015, Month=07"
"From=ITMTo=NGSDate=20150831Class=Y","Year=2015, Month=08"
"From=NGSTo=ITMDate=20150831Class=Y","Year=2015, Month=08"
"From=ITMTo=NGSDate=20150831Class=Y","Year=2015, Month=08"
"From=NGSTo=ITMDate=20150831Class=Y","Year=2015, Month=08"
"From=ITMTo=NGSDate=20150831Class=Y","Year=2015, Month=08"
"From=ITMTo=NGSDate=20150831Class=Y","Year=2015, Month=08"
"From=ITMTo=KMIDate=20150729Class=Y","Year=2015, Month=07"
"From=ITMTo=KOJDate=20150826Class=Y","Year=2015, Month=08"
"From=ITMTo=FUKDate=20150416Class=Y","Year=2015, Month=04"
"From=ITMTo=FUKDate=20151121Class=Y","Year=2015, Month=11"
"From=ITMTo=FUKDate=20151017Class=Y","Year=2015, Month=10"
"From=ITMTo=FUKDate=20151017Class=Y","Year=2015, Month=10"
"From=ITMTo=HNDDate=20151119Class=Y","Year=2015, Month=11"
"From=ITMTo=HNDDate=20151119Class=Y","Year=2015, Month=11"
"From=ITMTo=HNDDate=20151119Class=Y","Year=2015, Month=11"
"From=HNDTo=ITMDate=20151030Class=Y","Year=2015, Month=1

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Multiple Variables

Until this point, we've covered how to extract and map substring that matches patterns using one variable. This limits us to only one pattern extraction per column / string constant. <br>
Let's say we'd like to extract multiple substrings from one columns, such as 
- origin
- destination
- year
- month
- date


We can do this by using multiple variables, and assigning each patterns to them. For simplicity, we'll start with 2 variables, in order to extract origin and destination.<br>
Basic rule does not change when using multiple variables. You can fit them into one MapFrom and MapTo string. For example, to extract origin and destination, 

```From=%%ORG:@b:3-3%%To=%%DEST:@b:3-3%%%*```

And we'll be mapping both values in a single column for now in a format of `Origin=airport_code, Destination=airport_code`

Let's see this in action.

In [21]:
aq_pp -f,+1 $file -d $col -mapf search 'From=%%ORG:@b:3-3%%To=%%DEST:@b:3-3%%%*' \
-mapc S:airports 'Origin=%%ORG%%, Destination=%%DEST%%'

"search","airports"
"From=HNDTo=NGODate=20150506Class=Y","Origin=HND, Destination=NGO"
"From=NGik0To=OKADate=20150425Class=1","Origin=, Destination="
"From=OKATo=NGODate=20l5o4A5Class=Y","Origin=OKA, Destination=NGO"
"From=OKATo=NGODate=20150425Class=S","Origin=OKA, Destination=NGO"
"From=OKATo=nG0ODate=20150425Class=3","Origin=, Destination="
"From=3GOTo=OKADate=S0150F25Class=Y","Origin=, Destination="
"From=NGOTo=OKADate=20150419Class=Y","Origin=NGO, Destination=OKA"
"From=OKaTo=NGODate=20150517Class=Y","Origin=, Destination="
"From=OKATo=NGODate=20150517Class=Y","Origin=OKA, Destination=NGO"
"From=OKATo=NGODate=20150517Class=Y","Origin=OKA, Destination=NGO"
"From=OKATo=NGODate=20150418Class=Y","Origin=OKA, Destination=NGO"
"From=HNDTo=NGODate=20150815Class=Y","Origin=HND, Destination=NGO"
"From=HNDTo=NGODate=20150815Class=Y","Origin=HND, Destination=NGO"
"From=NGOTo=HNDDate=20150815Class=Y","Origin=NGO, Destination=HND"
"From=NGOTo=HNDDate=20150815Class=Y","Origin=NGO, Destination=H

"From=NGOTo=SPKDate=20160207Class=S","Origin=NGO, Destination=SPK"
"From=NGOTo=SPKDate=20160207Class=S","Origin=NGO, Destination=SPK"
"From=NGOTo=SPKDate=20160207Class=Y","Origin=NGO, Destination=SPK"
"From=NGOTo=SPKDate=20160207Class=Y","Origin=NGO, Destination=SPK"
"From=NGOTo=SPKDate=20160207Class=Y","Origin=NGO, Destination=SPK"
"From=NGOTo=SPKDate=20160207Class=Y","Origin=NGO, Destination=SPK"
"From=SPKTo=NGODate=20160207Class=S","Origin=SPK, Destination=NGO"
"From=SPKTo=NGODate=20160207Class=S","Origin=SPK, Destination=NGO"
"From=SPKTo=NGODate=20160207Class=Y","Origin=SPK, Destination=NGO"
"From=SPKTo=NGODate=20160207Class=S","Origin=SPK, Destination=NGO"
"From=SPKTo=NGODate=20160207Class=S","Origin=SPK, Destination=NGO"
"From=SPKTo=NGODate=20160207Class=S","Origin=SPK, Destination=NGO"
"From=SPKTo=NGODate=20160207Class=S","Origin=SPK, Destination=NGO"
"From=SPKTo=NGODate=20160207Class=Y","Origin=SPK, Destination=NGO"
"From=SPKTo=NGODate=20160207Class=S","Origin=SPK, Destination=

"From=SPKTo=NGODate=20160207Class=Y","Origin=SPK, Destination=NGO"
"From=SPKTo=NGODate=20160207Class=Y","Origin=SPK, Destination=NGO"
"From=NGOTo=SDJDate=20150926Class=Y","Origin=NGO, Destination=SDJ"
"From=NGOTo=SDJDate=20150926Class=Y","Origin=NGO, Destination=SDJ"
"From=SDJTo=NGODate=20150926Class=Y","Origin=SDJ, Destination=NGO"
"From=SDJTo=NGODate=20150927Class=Y","Origin=SDJ, Destination=NGO"
"From=SDJTo=NGODate=20150927Class=Y","Origin=SDJ, Destination=NGO"
"From=SDJTo=NGODate=20150926Class=Y","Origin=SDJ, Destination=NGO"
"From=NGOTo=SDJDate=20150926Class=Y","Origin=NGO, Destination=SDJ"
"From=NGOTo=SDJDate=20151204Class=Y","Origin=NGO, Destination=SDJ"
"From=NGOTo=SDJDate=20151205Class=Y","Origin=NGO, Destination=SDJ"
"From=NGOTo=SDJDate=20151206Class=Y","Origin=NGO, Destination=SDJ"
"From=SDJTo=NGODate=20151206Class=Y","Origin=SDJ, Destination=NGO"
"From=NGOTo=HNDDate=20160106Class=Y","Origin=NGO, Destination=HND"
"From=HNDTo=NGODate=20160106Class=Y","Origin=HND, Destination=

"From=NGOTo=ISGDate=20160528Class=Y","Origin=NGO, Destination=ISG"
"From=NGOTo=ISGDate=20160528Class=Y","Origin=NGO, Destination=ISG"
"From=NGOTo=ISGDate=20160528Class=Y","Origin=NGO, Destination=ISG"
"From=SPKTo=OKADate=20160529Class=Y","Origin=SPK, Destination=OKA"
"From=SPKTo=OKADate=20160529Class=Y","Origin=SPK, Destination=OKA"
"From=SPKTo=OKADate=20160529Class=Y","Origin=SPK, Destination=OKA"
"From=SPKTo=OKADate=20160529Class=S","Origin=SPK, Destination=OKA"
"From=SPKTo=OKADate=20160528Class=S","Origin=SPK, Destination=OKA"
"From=SPKTo=OKADate=20160528Class=Y","Origin=SPK, Destination=OKA"
"From=SPKTo=OKADate=20160528Class=S","Origin=SPK, Destination=OKA"
"From=SPKTo=OKADate=20160529Class=S","Origin=SPK, Destination=OKA"
"From=SPKTo=OKADate=20160529Class=Y","Origin=SPK, Destination=OKA"
"From=SPKTo=OKADate=20160521Class=Y","Origin=SPK, Destination=OKA"
"From=SPKTo=OKADate=20160521Class=S","Origin=SPK, Destination=OKA"
"From=SPKTo=OKADate=20160521Class=Y","Origin=SPK, Destination=

"From=ISGTo=NGODate=20160530Class=Y","Origin=ISG, Destination=NGO"
"From=ISGTo=NGODate=20160530Class=Y","Origin=ISG, Destination=NGO"
"From=ISGTo=NGODate=20160530Class=Y","Origin=ISG, Destination=NGO"
"From=ISGTo=NGODate=20160529Class=Y","Origin=ISG, Destination=NGO"
"From=ISGTo=NGODate=20160529Class=S","Origin=ISG, Destination=NGO"
"From=NGOTo=ISGDate=20160529Class=S","Origin=NGO, Destination=ISG"
"From=NGOTo=ISGDate=20160528Class=S","Origin=NGO, Destination=ISG"
"From=NGOTo=ISGDate=20160528Class=Y","Origin=NGO, Destination=ISG"
"From=NGOTo=OKADate=20160528Class=Y","Origin=NGO, Destination=OKA"
"From=NGOTo=OKADate=20160528Class=Y","Origin=NGO, Destination=OKA"
"From=NGOTo=OKADate=20160528Class=Y","Origin=NGO, Destination=OKA"
"From=NGOTo=OKADate=20160528Class=S","Origin=NGO, Destination=OKA"
"From=NGOTo=OKADate=20160528Class=S","Origin=NGO, Destination=OKA"
"From=NGOTo=OKADate=20160528Class=S","Origin=NGO, Destination=OKA"
"From=NGOTo=OKADate=20160529Class=S","Origin=NGO, Destination=

...

"From=OKATo=HNDDate=20180927Class=S","Origin=OKA, Destination=HND"
"From=OKATo=HNDDate=20180920Class=Y","Origin=OKA, Destination=HND"
"From=OKATo=HNDDate=20180926Class=S","Origin=OKA, Destination=HND"
"From=OKATo=HNDDate=20180927Class=S","Origin=OKA, Destination=HND"
"From=HNDTo=OKADate=20180926Class=Y","Origin=HND, Destination=OKA"
"From=OKATo=HNDDate=20180927Class=S","Origin=OKA, Destination=HND"
"From=HNDTo=OKADate=20180926Class=Y","Origin=HND, Destination=OKA"
"From=HNDTo=OKADate=20180920Class=Y","Origin=HND, Destination=OKA"
"From=HNDTo=OKADate=20180920Class=Y","Origin=HND, Destination=OKA"
"From=OKATo=HNDDate=20180927Class=Y","Origin=OKA, Destination=HND"
"From=OKATo=HNDDate=20180927Class=S","Origin=OKA, Destination=HND"
"From=HNDTo=OKADate=20180925Class=S","Origin=HND, Destination=OKA"
"From=HNDTo=SPKDate=20181007Class=Y","Origin=HND, Destination=SPK"
"From=HNDTo=SPKDate=20181007Class=Y","Origin=HND, Destination=SPK"
"From=HNDTo=FUKDate=20181024Class=Y","Origin=HND, Destination=

"From=SPKTo=HNDDate=20180102Class=S","Origin=SPK, Destination=HND"
"From=OSATo=HNDDate=20180123Class=Y","Origin=OSA, Destination=HND"
"From=OSATo=HNDDate=20180124Class=Y","Origin=OSA, Destination=HND"
"From=SDJTo=OSADate=20180122Class=Y","Origin=SDJ, Destination=OSA"
"From=OSATo=SPKDate=20180123Class=Y","Origin=OSA, Destination=SPK"
"From=OSATo=SPKDate=20180123Class=Y","Origin=OSA, Destination=SPK"
"From=AXTTo=HNDDate=20180305Class=Y","Origin=AXT, Destination=HND"
"From=AXTTo=HNDDate=20180304Class=Y","Origin=AXT, Destination=HND"
"From=AXTTo=SPKDate=20180304Class=Y","Origin=AXT, Destination=SPK"
"From=HNDTo=OSADate=20180301Class=Y","Origin=HND, Destination=OSA"
"From=ITMTo=AXTDate=20180304Class=Y","Origin=ITM, Destination=AXT"
"From=AXTTo=HNDDate=20180305Class=Y","Origin=AXT, Destination=HND"
"From=SPKTo=SDJDate=20180217Class=Y","Origin=SPK, Destination=SDJ"
"From=AXTTo=HNDDate=20180305Class=S","Origin=AXT, Destination=HND"
"From=SDJTo=SPKDate=20180326Class=Y","Origin=SDJ, Destination=

"From=HNDTo=SPKDate=20190101Class=Y","Origin=HND, Destination=SPK"
"From=SPKTo=HKDDate=20190102Class=Y","Origin=SPK, Destination=HKD"
"From=HKDTo=HNDDate=20190103Class=Y","Origin=HKD, Destination=HND"
"From=SPKTo=HNDDate=20190103Class=Y","Origin=SPK, Destination=HND"
"From=HKDTo=HNDDate=20190103Class=Y","Origin=HKD, Destination=HND"
"From=HNDTo=KMQDate=20190113Class=Y","Origin=HND, Destination=KMQ"
"From=KMQTo=HNDDate=20190114Class=Y","Origin=KMQ, Destination=HND"
"From=HNDTo=OSADate=20181207Class=Y","Origin=HND, Destination=OSA"
"From=HNDTo=OSADate=20181207Class=S","Origin=HND, Destination=OSA"
"From=HNDTo=OSADate=20181207Class=Y","Origin=HND, Destination=OSA"
"From=OSATo=HNDDate=20181209Class=Y","Origin=OSA, Destination=HND"
"From=KMQTo=HNDDate=20190114Class=Y","Origin=KMQ, Destination=HND"
"From=HNDTo=SPKDate=20190101Class=Y","Origin=HND, Destination=SPK"
"From=HNDTo=SPKDate=20190101Class=S","Origin=HND, Destination=SPK"
"From=HNDTo=SPKDate=20190101Class=Y","Origin=HND, Destination=

"From=ITMTo=NGSDate=20150831Class=Y","Origin=ITM, Destination=NGS"
"From=NGSTo=ITMDate=20150831Class=Y","Origin=NGS, Destination=ITM"
"From=ITMTo=NGSDate=20150831Class=Y","Origin=ITM, Destination=NGS"
"From=NGSTo=ITMDate=20150831Class=Y","Origin=NGS, Destination=ITM"
"From=ITMTo=NGSDate=20150831Class=Y","Origin=ITM, Destination=NGS"
"From=ITMTo=NGSDate=20150831Class=Y","Origin=ITM, Destination=NGS"
"From=ITMTo=KMIDate=20150729Class=Y","Origin=ITM, Destination=KMI"
"From=ITMTo=KOJDate=20150826Class=Y","Origin=ITM, Destination=KOJ"
"From=ITMTo=FUKDate=20150416Class=Y","Origin=ITM, Destination=FUK"
"From=ITMTo=FUKDate=20151121Class=Y","Origin=ITM, Destination=FUK"
"From=ITMTo=FUKDate=20151017Class=Y","Origin=ITM, Destination=FUK"
"From=ITMTo=FUKDate=20151017Class=Y","Origin=ITM, Destination=FUK"
"From=ITMTo=HNDDate=20151119Class=Y","Origin=ITM, Destination=HND"
"From=ITMTo=HNDDate=20151119Class=Y","Origin=ITM, Destination=HND"
"From=ITMTo=HNDDate=20151119Class=Y","Origin=ITM, Destination=

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Let's scale up to extracting 3 substrings, origin, destination and date (year, month and date).

For extraction, we can simply combine 3 variables together in MapFrom, like following.

`From=%%ORG:@b:3-3%%To=%%DEST:@b:3-3%%Date=%%TIME:@n:8-8%%%*`

And Map it in format of 

`Origin=airport_code, Destination=airport_code, date=date_digits`

In [22]:
aq_pp -f,+1 $file -d $col -mapf search 'From=%%ORG:@b:3-3%%To=%%DEST:@b:3-3%%Date=%%TIME:@n:8-8%%%*' \
-mapc S:airports 'Origin=%%ORG%%, Destination=%%DEST%%, Date=%%TIME%%'

"search","airports"
"From=HNDTo=NGODate=20150506Class=Y","Origin=HND, Destination=NGO, Date=20150506"
"From=NGik0To=OKADate=20150425Class=1","Origin=, Destination=, Date="
"From=OKATo=NGODate=20l5o4A5Class=Y","Origin=, Destination=, Date="
"From=OKATo=NGODate=20150425Class=S","Origin=OKA, Destination=NGO, Date=20150425"
"From=OKATo=nG0ODate=20150425Class=3","Origin=, Destination=, Date="
"From=3GOTo=OKADate=S0150F25Class=Y","Origin=, Destination=, Date="
"From=NGOTo=OKADate=20150419Class=Y","Origin=NGO, Destination=OKA, Date=20150419"
"From=OKaTo=NGODate=20150517Class=Y","Origin=, Destination=, Date="
"From=OKATo=NGODate=20150517Class=Y","Origin=OKA, Destination=NGO, Date=20150517"
"From=OKATo=NGODate=20150517Class=Y","Origin=OKA, Destination=NGO, Date=20150517"
"From=OKATo=NGODate=20150418Class=Y","Origin=OKA, Destination=NGO, Date=20150418"
"From=HNDTo=NGODate=20150815Class=Y","Origin=HND, Destination=NGO, Date=20150815"
"From=HNDTo=NGODate=20150815Class=Y","Origin=HND, Destination=N

"From=NGOTo=SDJDate=20150926Class=Y","Origin=NGO, Destination=SDJ, Date=20150926"
"From=NGOTo=SDJDate=20150926Class=Y","Origin=NGO, Destination=SDJ, Date=20150926"
"From=NGOTo=SDJDate=20150926Class=Y","Origin=NGO, Destination=SDJ, Date=20150926"
"From=NGOTo=SDJDate=20150926Class=Y","Origin=NGO, Destination=SDJ, Date=20150926"
"From=NGOTo=SDJDate=20150828Class=Y","Origin=NGO, Destination=SDJ, Date=20150828"
"From=NGOTo=SDJDate=20150829Class=Y","Origin=NGO, Destination=SDJ, Date=20150829"
"From=NGOTo=SDJDate=20150830Class=Y","Origin=NGO, Destination=SDJ, Date=20150830"
"From=SDJTo=NGODate=20150830Class=Y","Origin=SDJ, Destination=NGO, Date=20150830"
"From=SDJTo=NGODate=20150830Class=Y","Origin=SDJ, Destination=NGO, Date=20150830"
"From=SPKTo=NGODate=20160207Class=S","Origin=SPK, Destination=NGO, Date=20160207"
"From=SPKTo=NGODate=20160207Class=S","Origin=SPK, Destination=NGO, Date=20160207"
"From=SPKTo=NGODate=20160207Class=S","Origin=SPK, Destination=NGO, Date=20160207"
"From=SPKTo=NGOD

"From=NGOTo=SDJDate=20150927Class=Y","Origin=NGO, Destination=SDJ, Date=20150927"
"From=SDJTo=NGODate=20150927Class=Y","Origin=SDJ, Destination=NGO, Date=20150927"
"From=SDJTo=NGODate=20150926Class=Y","Origin=SDJ, Destination=NGO, Date=20150926"
"From=NGOTo=SDJDate=20150926Class=Y","Origin=NGO, Destination=SDJ, Date=20150926"
"From=NGOTo=SDJDate=20150926Class=Y","Origin=NGO, Destination=SDJ, Date=20150926"
"From=SDJTo=NGODate=20150926Class=Y","Origin=SDJ, Destination=NGO, Date=20150926"
"From=SDJTo=NGODate=20150927Class=Y","Origin=SDJ, Destination=NGO, Date=20150927"
"From=NGOTo=SDJDate=20150927Class=Y","Origin=NGO, Destination=SDJ, Date=20150927"
"From=NGOTo=SDJDate=20150926Class=Y","Origin=NGO, Destination=SDJ, Date=20150926"
"From=NGOTo=SDJDate=20150925Class=Y","Origin=NGO, Destination=SDJ, Date=20150925"
"From=NGOTo=SDJDate=20150926Class=Y","Origin=NGO, Destination=SDJ, Date=20150926"
"From=NGOTo=SDJDate=20150924Class=Y","Origin=NGO, Destination=SDJ, Date=20150924"
"From=NGOTo=HNDD

"From=NGOTo=ISGDate=20160319Class=S","Origin=NGO, Destination=ISG, Date=20160319"
"From=NGOTo=ISGDate=20160319Class=S","Origin=NGO, Destination=ISG, Date=20160319"
"From=NGOTo=ISGDate=20160319Class=S","Origin=NGO, Destination=ISG, Date=20160319"
"From=NGOTo=ISGDate=20160319Class=Y","Origin=NGO, Destination=ISG, Date=20160319"
"From=HNDTo=ISGDate=20160319Class=Y","Origin=HND, Destination=ISG, Date=20160319"
"From=HNDTo=ISGDate=20160319Class=S","Origin=HND, Destination=ISG, Date=20160319"
"From=HNDTo=ISGDate=20160319Class=Y","Origin=HND, Destination=ISG, Date=20160319"
"From=HNDTo=ISGDate=20160319Class=S","Origin=HND, Destination=ISG, Date=20160319"
"From=HNDTo=ISGDate=20160319Class=Y","Origin=HND, Destination=ISG, Date=20160319"
"From=HNDTo=ISGDate=20160416Class=Y","Origin=HND, Destination=ISG, Date=20160416"
"From=HNDTo=ISGDate=20160423Class=Y","Origin=HND, Destination=ISG, Date=20160423"
"From=HNDTo=ISGDate=20160423Class=Y","Origin=HND, Destination=ISG, Date=20160423"
"From=HNDTo=ISGD

"From=ISGTo=NGODate=20160528Class=Y","Origin=ISG, Destination=NGO, Date=20160528"
"From=NGOTo=ISGDate=20160528Class=Y","Origin=NGO, Destination=ISG, Date=20160528"
"From=ISGTo=NGODate=20160528Class=Y","Origin=ISG, Destination=NGO, Date=20160528"
"From=NGOTo=ISGDate=20160528Class=Y","Origin=NGO, Destination=ISG, Date=20160528"
"From=ISGTo=NGODate=20160528Class=Y","Origin=ISG, Destination=NGO, Date=20160528"
"From=NGOTo=ISGDate=20160528Class=Y","Origin=NGO, Destination=ISG, Date=20160528"
"From=NGOTo=OKADate=20160528Class=Y","Origin=NGO, Destination=OKA, Date=20160528"
"From=NGOTo=OKADate=20160528Class=Y","Origin=NGO, Destination=OKA, Date=20160528"
"From=NGOTo=OKADate=20160528Class=Y","Origin=NGO, Destination=OKA, Date=20160528"
"From=NGOTo=OKADate=20160528Class=S","Origin=NGO, Destination=OKA, Date=20160528"
"From=NGOTo=OKADate=20160528Class=S","Origin=NGO, Destination=OKA, Date=20160528"
"From=NGOTo=OKADate=20160528Class=S","Origin=NGO, Destination=OKA, Date=20160528"
"From=OKATo=NGOD

...

"From=HNDTo=SPKDate=20180808Class=Y","Origin=HND, Destination=SPK, Date=20180808"
"From=HNDTo=SPKDate=20180809Class=Y","Origin=HND, Destination=SPK, Date=20180809"
"From=HNDTo=SPKDate=20180810Class=Y","Origin=HND, Destination=SPK, Date=20180810"
"From=HNDTo=SPKDate=20180810Class=Y","Origin=HND, Destination=SPK, Date=20180810"
"From=HNDTo=SPKDate=20180810Class=Y","Origin=HND, Destination=SPK, Date=20180810"
"From=HNDTo=SPKDate=20180810Class=Y","Origin=HND, Destination=SPK, Date=20180810"
"From=HNDTo=SPKDate=20180810Class=S","Origin=HND, Destination=SPK, Date=20180810"
"From=HNDTo=SPKDate=20180810Class=Y","Origin=HND, Destination=SPK, Date=20180810"
"From=HNDTo=SPKDate=20180810Class=S","Origin=HND, Destination=SPK, Date=20180810"
"From=HNDTo=OKADate=20180919Class=Y","Origin=HND, Destination=OKA, Date=20180919"
"From=HNDTo=OKADate=20180919Class=Y","Origin=HND, Destination=OKA, Date=20180919"
"From=OKATo=HNDDate=20180920Class=Y","Origin=OKA, Destination=HND, Date=20180920"
"From=HNDTo=OKAD

"From=HNDTo=ITMDate=20181207Class=S","Origin=HND, Destination=ITM, Date=20181207"
"From=HNDTo=ITMDate=20181207Class=Y","Origin=HND, Destination=ITM, Date=20181207"
"From=HNDTo=SPKDate=20190101Class=S","Origin=HND, Destination=SPK, Date=20190101"
"From=HNDTo=SPKDate=20190101Class=S","Origin=HND, Destination=SPK, Date=20190101"
"From=HNDTo=KMQDate=20190113Class=Y","Origin=HND, Destination=KMQ, Date=20190113"
"From=HNDTo=KMQDate=20190113Class=Y","Origin=HND, Destination=KMQ, Date=20190113"
"From=HNDTo=KMQDate=20190113Class=Y","Origin=HND, Destination=KMQ, Date=20190113"
"From=HNDTo=KMQDate=20190113Class=Y","Origin=HND, Destination=KMQ, Date=20190113"
"From=HNDTo=SPKDate=20190101Class=S","Origin=HND, Destination=SPK, Date=20190101"
"From=HNDTo=SPKDate=20190101Class=S","Origin=HND, Destination=SPK, Date=20190101"
"From=KIXTo=HNDDate=20181209Class=Y","Origin=KIX, Destination=HND, Date=20181209"
"From=ITMTo=HNDDate=20181209Class=Y","Origin=ITM, Destination=HND, Date=20181209"
"From=HNDTo=ITMD

"From=HNDTo=OSADate=20180714Class=Y","Origin=HND, Destination=OSA, Date=20180714"
"From=HNDTo=OSADate=20180714Class=S","Origin=HND, Destination=OSA, Date=20180714"
"From=HNDTo=OSADate=20180714Class=Y","Origin=HND, Destination=OSA, Date=20180714"
"From=HNDTo=OSADate=20180713Class=Y","Origin=HND, Destination=OSA, Date=20180713"
"From=OSATo=HNDDate=20180714Class=Y","Origin=OSA, Destination=HND, Date=20180714"
"From=OSATo=HNDDate=20180714Class=S","Origin=OSA, Destination=HND, Date=20180714"
"From=HNDTo=OSADate=20180714Class=S","Origin=HND, Destination=OSA, Date=20180714"
"From=OSATo=HNDDate=20180715Class=S","Origin=OSA, Destination=HND, Date=20180715"
"From=HNDTo=OSADate=20180714Class=Y","Origin=HND, Destination=OSA, Date=20180714"
"From=HNDTo=OSADate=20180714Class=S","Origin=HND, Destination=OSA, Date=20180714"
"From=OSATo=HNDDate=20180716Class=S","Origin=OSA, Destination=HND, Date=20180716"
"From=OSATo=HNDDate=20180716Class=Y","Origin=OSA, Destination=HND, Date=20180716"
"From=OSATo=HNDD

"From=HNDTo=KMQDate=20190113Class=Y","Origin=HND, Destination=KMQ, Date=20190113"
"From=HNDTo=SPKDate=20190101Class=Y","Origin=HND, Destination=SPK, Date=20190101"
"From=HNDTo=SPKDate=20190101Class=S","Origin=HND, Destination=SPK, Date=20190101"
"From=HNDTo=OSADate=20181207Class=Y","Origin=HND, Destination=OSA, Date=20181207"
"From=OSATo=HNDDate=20181209Class=Y","Origin=OSA, Destination=HND, Date=20181209"
"From=HNDTo=SPKDate=20190101Class=Y","Origin=HND, Destination=SPK, Date=20190101"
"From=HNDTo=SPKDate=20190101Class=Y","Origin=HND, Destination=SPK, Date=20190101"
"From=HNDTo=FUKDate=20150418Class=Y","Origin=HND, Destination=FUK, Date=20150418"
"From=HNDTo=FUKDate=20150418Class=Y","Origin=HND, Destination=FUK, Date=20150418"
"From=HNDTo=FUKDate=20150418Class=Y","Origin=HND, Destination=FUK, Date=20150418"
"From=HNDTo=FUKDate=20150418Class=Y","Origin=HND, Destination=FUK, Date=20150418"
"From=HNDTo=KIXDate=20150530Class=Y","Origin=HND, Destination=KIX, Date=20150530"
"From=HNDTo=ITMD

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Mapping to Multiple Columns

Map each variables for each columns.